<a href="https://colab.research.google.com/github/AkshChkrbrty/ISDQuantumVersions/blob/main/LeeAndBrickell_S_ISD_QuantumVersion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install qiskit matplotlib
!pip install qiskit
!pip install qiskit_ibm_runtime
!pip install matplotlib
!pip install pylatexenc
!pip install qiskit_aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.2/366.2 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.8/75.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 10.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.3
    Uninstalling requests-2.32.3:
      Successfully uninstalled requests-2.32.3
  Attempting uninstall: PyJWT
    Found existing i

In [ ]:
import math
from math import pi, asin, sqrt, floor
from qiskit import QuantumCircuit
from qiskit.circuit.library import GroverOperator, MCMT, ZGate
from qiskit.visualization import plot_distribution
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.library import GroverOperator
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit_ibm_runtime import Session as Session
from itertools import combinations
import random

In [ ]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.library import QFT
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Statevector
from qiskit.circuit.library import MCXGate
import random
from math import pi, sqrt, floor, log2, ceil
from itertools import combinations
from collections import defaultdict

class EnhancedQuantumLeeBrickell:
    def __init__(self, H, syndrome, target_weight, p_weight):
        """
        Enhanced Quantum Lee-Brickell decoder with improved quantum components

        Args:
            H: Parity check matrix (numpy array)
            syndrome: Error syndrome (numpy array)
            target_weight: Total error weight t
            p_weight: Weight of e2 portion (p)
        """
        self.H = H
        self.syndrome = syndrome
        self.n, self.k = H.shape[1], H.shape[1] - H.shape[0]
        self.m = H.shape[0]
        self.target_weight = target_weight
        self.p_weight = p_weight
        self.e1_weight = target_weight - p_weight

        # Quantum preprocessing results
        self.X = None
        self.s_prime = None
        self.P = None
        self.preprocessing_success = False

    def quantum_preprocessing(self, max_attempts=10):
        """
        Quantum-enhanced preprocessing with multiple permutation attempts
        """
        preprocessing_circuit = self.create_preprocessing_circuit(max_attempts)

        # Simulate preprocessing
        simulator = AerSimulator()
        job = simulator.run(preprocessing_circuit, shots=1024)
        result = job.result()
        counts = result.get_counts()

        # Extract best permutation result
        success = self.extract_preprocessing_result(counts)
        self.preprocessing_success = success
        return success

    def create_preprocessing_circuit(self, max_attempts):
        """
        Create quantum circuit for preprocessing with superposition of permutations
        """
        # Use log2(max_attempts) qubits to encode permutation choice
        perm_qubits = max(1, ceil(log2(max_attempts)))
        perm_reg = QuantumRegister(perm_qubits, 'perm')
        result_reg = ClassicalRegister(perm_qubits, 'result')

        qc = QuantumCircuit(perm_reg, result_reg)

        # Create superposition over permutation choices
        qc.h(perm_reg)

        # For simplicity in this demonstration, we'll measure and use classical post-processing
        # In a full quantum implementation, this would involve quantum linear algebra
        qc.measure(perm_reg, result_reg)

        return qc

    def extract_preprocessing_result(self, counts):
        """
        Extract the best preprocessing result from quantum measurements
        """
        # Try different permutations based on measurement outcomes
        for bitstring, count in sorted(counts.items(), key=lambda x: x[1], reverse=True):
            perm_seed = int(bitstring, 2)

            if self.try_permutation(perm_seed):
                return True

        return False

    def try_permutation(self, seed):
        """
        Try a specific permutation for preprocessing
        """
        random.seed(seed)
        perm_indices = list(range(self.n))
        random.shuffle(perm_indices)
        self.P = np.eye(self.n, dtype=int)[perm_indices]

        # Apply permutation and Gaussian elimination
        H_perm = self.H @ self.P
        return self.gaussian_elimination(H_perm)

    def gaussian_elimination(self, H_perm):
        """
        Perform Gaussian elimination to get systematic form
        """
        H_work = H_perm.copy()
        pivot_cols = []

        for row in range(min(self.m, self.n)):
            pivot_col = None
            for col in range(self.n):
                if col not in pivot_cols and H_work[row, col] == 1:
                    pivot_col = col
                    break

            if pivot_col is None:
                continue

            pivot_cols.append(pivot_col)

            for other_row in range(self.m):
                if other_row != row and H_work[other_row, pivot_col] == 1:
                    H_work[other_row] = (H_work[other_row] + H_work[row]) % 2

        if len(pivot_cols) < self.m:
            return False

        # Extract systematic form components
        info_cols = [col for col in range(self.n) if col not in pivot_cols[:self.m]]

        if len(info_cols) != self.k:
            return False

        self.X = H_work[:self.m, info_cols]
        self.s_prime = self.syndrome.copy()
        return True

    def create_quantum_weight_oracle(self, qreg, target_weight, ancilla_reg):
        """
        Enhanced quantum weight checking oracle using QFT-based approach
        """
        qc = QuantumCircuit(qreg, ancilla_reg)
        n_qubits = len(qreg)

        if target_weight == 0:
            # All qubits must be |0⟩
            for i in range(n_qubits):
                qc.x(qreg[i])
            qc.mcx(qreg, ancilla_reg[0])
            for i in range(n_qubits):
                qc.x(qreg[i])

        elif target_weight == n_qubits:
            # All qubits must be |1⟩
            qc.mcx(qreg, ancilla_reg[0])

        else:
            # Use QFT-based weight checking for intermediate weights
            self.qft_weight_check(qc, qreg, target_weight, ancilla_reg)

        return qc

    def qft_weight_check(self, qc, qreg, target_weight, ancilla_reg):
        """
        QFT-based weight checking for more efficient implementation
        """
        n_qubits = len(qreg)

        # Create auxiliary register for counting
        count_bits = ceil(log2(n_qubits + 1))
        if len(ancilla_reg) < count_bits + 1:
            # Simplified approach for limited ancilla
            self.enumerate_weight_check(qc, qreg, target_weight, ancilla_reg[0])
            return

        count_reg = ancilla_reg[:count_bits]
        result_reg = ancilla_reg[count_bits]

        # Quantum adder to count set bits
        for i, qubit in enumerate(qreg):
            self.quantum_increment(qc, qubit, count_reg)

        # Check if count equals target_weight
        self.check_count_value(qc, count_reg, target_weight, result_reg)

        # Uncompute the counting
        for i, qubit in enumerate(reversed(qreg)):
            self.quantum_decrement(qc, qubit, count_reg)

    def quantum_increment(self, qc, control, count_reg):
        """
        Quantum increment controlled by control qubit
        """
        for i in range(len(count_reg)):
            if i == 0:
                qc.cx(control, count_reg[i])
            else:
                # Controlled increment with carry
                controls = [control] + count_reg[:i]
                qc.mcx(controls, count_reg[i])

    def quantum_decrement(self, qc, control, count_reg):
        """
        Quantum decrement (inverse of increment)
        """
        for i in range(len(count_reg)-1, -1, -1):
            if i == 0:
                qc.cx(control, count_reg[i])
            else:
                controls = [control] + count_reg[:i]
                qc.mcx(controls, count_reg[i])

    def check_count_value(self, qc, count_reg, target_value, result_qubit):
        """
        Check if count register contains target value
        """
        # Convert target_value to binary
        target_bits = format(target_value, f'0{len(count_reg)}b')

        # Flip qubits that should be 0 in target
        for i, bit in enumerate(target_bits):
            if bit == '0':
                qc.x(count_reg[i])

        # Multi-controlled X to result
        qc.mcx(count_reg, result_qubit)

        # Unflip the qubits
        for i, bit in enumerate(target_bits):
            if bit == '0':
                qc.x(count_reg[i])

    def enumerate_weight_check(self, qc, qreg, target_weight, result_qubit):
        """
        Fallback enumeration-based weight check for small cases
        """
        n_qubits = len(qreg)
        if n_qubits > 8:  # Practical limit
            return

        for combo in combinations(range(n_qubits), target_weight):
            # Flip qubits not in combination
            for i in range(n_qubits):
                if i not in combo:
                    qc.x(qreg[i])

            # Apply multi-controlled operation
            qc.mcx(qreg, result_qubit)

            # Unflip qubits
            for i in range(n_qubits):
                if i not in combo:
                    qc.x(qreg[i])

    def create_enhanced_oracle(self, e2_reg, ancilla_reg):
        """
        Enhanced oracle with both weight and syndrome constraints
        """
        qc = QuantumCircuit(e2_reg, ancilla_reg)

        # Ancilla allocation
        weight_anc = ancilla_reg[0]
        syndrome_anc = ancilla_reg[1] if len(ancilla_reg) > 1 else ancilla_reg[0]
        e1_reg = ancilla_reg[2:2+self.m] if len(ancilla_reg) > 2+self.m else []

        # Step 1: Check wt(e2) = p_weight
        weight_oracle = self.create_quantum_weight_oracle(e2_reg, self.p_weight, [weight_anc])
        qc.compose(weight_oracle, inplace=True)

        # Step 2: Compute e1 = s' + X * e2 (if we have enough ancilla)
        if len(e1_reg) >= self.m:
            self.quantum_gf2_matvec(qc, e2_reg, e1_reg)
            self.add_syndrome(qc, e1_reg)

            # Check wt(e1) = e1_weight
            e1_weight_oracle = self.create_quantum_weight_oracle(
                e1_reg, self.e1_weight, [syndrome_anc]
            )
            qc.compose(e1_weight_oracle, inplace=True)

            # Combined condition: both weights correct
            qc.ccz(weight_anc, syndrome_anc, ancilla_reg[-1])

            # Uncompute e1 computation
            e1_weight_oracle_inv = e1_weight_oracle.inverse()
            qc.compose(e1_weight_oracle_inv, inplace=True)
            self.add_syndrome_inv(qc, e1_reg)
            self.quantum_gf2_matvec_inv(qc, e2_reg, e1_reg)
        else:
            # Simplified oracle with just weight check
            qc.z(weight_anc)

        # Uncompute weight check
        weight_oracle_inv = weight_oracle.inverse()
        qc.compose(weight_oracle_inv, inplace=True)

        return qc

    def quantum_gf2_matvec(self, qc, vec_reg, result_reg):
        """
        Quantum GF(2) matrix-vector multiplication
        """
        for i in range(min(len(result_reg), self.X.shape[0])):
            for j in range(min(len(vec_reg), self.X.shape[1])):
                if self.X[i, j] == 1:
                    qc.cx(vec_reg[j], result_reg[i])

    def quantum_gf2_matvec_inv(self, qc, vec_reg, result_reg):
        """
        Inverse of quantum GF(2) matrix-vector multiplication
        """
        for i in range(min(len(result_reg), self.X.shape[0])-1, -1, -1):
            for j in range(min(len(vec_reg), self.X.shape[1])-1, -1, -1):
                if self.X[i, j] == 1:
                    qc.cx(vec_reg[j], result_reg[i])

    def add_syndrome(self, qc, e1_reg):
        """
        Add syndrome to e1 register
        """
        for i in range(min(len(e1_reg), len(self.s_prime))):
            if self.s_prime[i] == 1:
                qc.x(e1_reg[i])

    def add_syndrome_inv(self, qc, e1_reg):
        """
        Inverse syndrome addition
        """
        for i in range(min(len(e1_reg), len(self.s_prime))-1, -1, -1):
            if self.s_prime[i] == 1:
                qc.x(e1_reg[i])

    def create_amplitude_amplification_diffuser(self, qreg):
        """
        Enhanced diffuser using amplitude amplification principles
        """
        qc = QuantumCircuit(qreg)
        n_qubits = len(qreg)

        # Apply Hadamard gates
        qc.h(qreg)

        # Apply Z to |0...0⟩ state
        qc.x(qreg)

        # Multi-controlled Z gate
        if n_qubits == 1:
            qc.z(qreg[0])
        elif n_qubits == 2:
            qc.cz(qreg[0], qreg[1])
        else:
            qc.h(qreg[-1])
            qc.mcx(qreg[:-1], qreg[-1])
            qc.h(qreg[-1])

        qc.x(qreg)
        qc.h(qreg)

        return qc

    def estimate_solution_count(self):
        """
        Enhanced solution count estimation
        """
        from math import comb

        # Theoretical upper bound
        max_e2_combinations = comb(self.k, self.p_weight) if self.p_weight <= self.k else 0

        # Heuristic estimation based on code parameters
        # For random-like matrices, probability of correct e1 weight
        if self.e1_weight <= self.m:
            # Binomial approximation
            prob_factor = comb(self.m, self.e1_weight) / (2**self.m)
            estimated_solutions = max(1, int(max_e2_combinations * prob_factor))
        else:
            estimated_solutions = 0

        return min(estimated_solutions, max_e2_combinations)

    def adaptive_grover_iterations(self, total_space, estimated_solutions):
        """
        Adaptive iteration count for Grover's algorithm
        """
        if estimated_solutions == 0:
            return 5  # Minimal search

        # Optimal Grover iterations
        optimal = int(pi * sqrt(total_space / estimated_solutions) / 4)

        # Conservative bounds
        min_iter = max(1, optimal // 2)
        max_iter = min(optimal * 2, 50)  # Safety limit

        return min_iter + (max_iter - min_iter) // 2

    def quantum_search(self, shots=2048):
        """
        Enhanced quantum search with adaptive parameters
        """
        if not self.preprocessing_success:
            print("Preprocessing failed - cannot proceed with quantum search")
            return None

        # Create quantum registers
        e2_reg = QuantumRegister(self.k, 'e2')

        # Calculate required ancilla qubits
        min_ancilla = 3  # Basic oracle requirements
        weight_check_ancilla = ceil(log2(self.k + 1)) + 1
        matrix_ancilla = self.m
        total_ancilla = max(min_ancilla, weight_check_ancilla + matrix_ancilla + 2)

        ancilla_reg = QuantumRegister(total_ancilla, 'anc')
        result_reg = ClassicalRegister(self.k, 'result')

        qc = QuantumCircuit(e2_reg, ancilla_reg, result_reg)

        # Initialize superposition
        qc.h(e2_reg)

        # Calculate adaptive iteration count
        total_space = 2**self.k
        estimated_solutions = self.estimate_solution_count()
        iterations = self.adaptive_grover_iterations(total_space, estimated_solutions)

        print(f"Search parameters:")
        print(f"  Total space: 2^{self.k} = {total_space}")
        print(f"  Estimated solutions: {estimated_solutions}")
        print(f"  Grover iterations: {iterations}")
        print(f"  Ancilla qubits: {total_ancilla}")

        # Create oracle and diffuser
        oracle = self.create_enhanced_oracle(e2_reg, ancilla_reg)
        diffuser = self.create_amplitude_amplification_diffuser(e2_reg)

        # Grover iterations
        for i in range(iterations):
            qc.compose(oracle, inplace=True)
            qc.compose(diffuser, inplace=True)

        # Measurement
        qc.measure(e2_reg, result_reg)

        return qc

    def verify_quantum_solution(self, e2_bitstring):
        """
        Verify a candidate solution from quantum measurement
        """
        e2 = np.array([int(b) for b in e2_bitstring], dtype=int)

        # Check e2 weight
        if np.sum(e2) != self.p_weight:
            return False, None

        # Compute e1
        e1 = (self.s_prime + self.X @ e2) % 2

        # Check e1 weight
        if np.sum(e1) != self.e1_weight:
            return False, None

        # Reconstruct full error in systematic form
        e_systematic = np.concatenate([e1, e2])

        # Apply inverse permutation
        e_full = self.P.T @ e_systematic

        # Final verification
        computed_syndrome = (self.H @ e_full) % 2
        if np.array_equal(computed_syndrome, self.syndrome):
            return True, e_full

        return False, None

    def decode(self, shots=2048, max_quantum_attempts=3):
        """
        Main quantum decoding function
        """
        print("Enhanced Quantum Lee-Brickell Decoding")
        print("="*50)

        # Quantum preprocessing
        print("Phase 1: Quantum Preprocessing")
        if not self.quantum_preprocessing():
            print("❌ Quantum preprocessing failed")
            return None

        print("✅ Quantum preprocessing successful")
        print(f"   Systematic form X matrix shape: {self.X.shape}")
        print(f"   Modified syndrome s': {self.s_prime}")

        # Multiple quantum search attempts
        for attempt in range(max_quantum_attempts):
            print(f"\nPhase 2: Quantum Search (Attempt {attempt + 1})")

            try:
                qc = self.quantum_search(shots)
                if qc is None:
                    continue

                # Execute quantum circuit
                simulator = AerSimulator()
                job = simulator.run(qc, shots=shots)
                result = job.result()
                counts = result.get_counts()

                print(f"Top measurement outcomes:")
                top_results = sorted(counts.items(), key=lambda x: x[1], reverse=True)[:10]
                for bitstring, count in top_results:
                    print(f"  {bitstring}: {count} ({100*count/shots:.1f}%)")

                # Phase 3: Solution verification
                print(f"\nPhase 3: Solution Verification")
                for bitstring, count in top_results:
                    is_valid, error_vector = self.verify_quantum_solution(bitstring[::-1])

                    if is_valid:
                        print(f"✅ Valid solution found!")
                        print(f"   Error vector: {error_vector}")
                        print(f"   Weight: {np.sum(error_vector)}")
                        print(f"   Measurement frequency: {count}/{shots}")
                        return error_vector

                print(f"❌ No valid solution in attempt {attempt + 1}")

            except Exception as e:
                print(f"❌ Quantum search attempt {attempt + 1} failed: {e}")
                continue

        print(f"\n❌ All {max_quantum_attempts} quantum attempts failed")
        return None


def create_enhanced_test_case():
    """
    Create a comprehensive test case for the enhanced algorithm
    """
    print("Creating Enhanced Test Case")
    print("-" * 30)

    # Use [15,11,3] BCH code for more realistic testing
    # Simplified example - in practice would use proper BCH construction
    np.random.seed(42)  # For reproducibility

    # Create a random systematic parity check matrix
    n, k = 7, 4  # Keep it small for demonstration
    m = n - k

    # Random parity check matrix in systematic form [I | P]
    I = np.eye(m, dtype=int)
    P = np.random.randint(0, 2, (m, k))
    H = np.hstack([I, P])

    print(f"Code parameters: [{n},{k},{3}] (length, dimension, min distance)")
    print(f"Parity check matrix H ({m}×{n}):")
    print(H)

    # Create a test error with known structure
    target_weight = 2
    p_weight = 1

    # Construct error strategically
    true_error = np.zeros(n, dtype=int)
    true_error[1] = 1  # In parity section
    true_error[5] = 1  # In information section

    syndrome = (H @ true_error) % 2

    print(f"\nTest error configuration:")
    print(f"  True error: {true_error} (weight: {np.sum(true_error)})")
    print(f"  Target weight t: {target_weight}")
    print(f"  e2 weight p: {p_weight}")
    print(f"  e1 weight (t-p): {target_weight - p_weight}")
    print(f"  Syndrome: {syndrome}")

    decoder = EnhancedQuantumLeeBrickell(H, syndrome, target_weight, p_weight)
    return decoder, true_error


# Enhanced main execution
if __name__ == "__main__":
    print("Enhanced Quantum Lee-Brickell Information Set Decoding")
    print("=" * 60)

    # Create test case
    decoder, true_error = create_enhanced_test_case()

    # Run enhanced quantum decoding
    print(f"\nStarting Enhanced Quantum Decoding...")
    decoded_error = decoder.decode(shots=4096, max_quantum_attempts=2)

    # Results analysis
    print(f"\n" + "=" * 60)
    print("FINAL RESULTS")
    print("=" * 60)

    if decoded_error is not None:
        print("🎉 DECODING SUCCESSFUL!")
        print(f"   True error:    {true_error}")
        print(f"   Decoded error: {decoded_error}")
        print(f"   Exact match:   {np.array_equal(true_error, decoded_error)}")
        print(f"   Weight match:  {np.sum(true_error)} = {np.sum(decoded_error)}")

        # Verify syndrome
        computed_syndrome = (decoder.H @ decoded_error) % 2
        syndrome_match = np.array_equal(computed_syndrome, decoder.syndrome)
        print(f"   Syndrome check: {syndrome_match}")

    else:
        print("❌ DECODING FAILED")
        print("Possible improvements:")
        print("  • Increase shot count for better statistics")
        print("  • Adjust Grover iteration count")
        print("  • Try different weight parameter p")
        print("  • Use more sophisticated oracle design")
        print("  • Consider hybrid classical-quantum preprocessing")

Enhanced Quantum Lee-Brickell Information Set Decoding
Creating Enhanced Test Case
------------------------------
Code parameters: [7,4,3] (length, dimension, min distance)
Parity check matrix H (3×7):
[[1 0 0 0 1 0 0]
 [0 1 0 0 1 0 0]
 [0 0 1 0 1 0 0]]

Test error configuration:
  True error: [0 1 0 0 0 1 0] (weight: 2)
  Target weight t: 2
  e2 weight p: 1
  e1 weight (t-p): 1
  Syndrome: [0 1 0]

Starting Enhanced Quantum Decoding...
Enhanced Quantum Lee-Brickell Decoding
Phase 1: Quantum Preprocessing
✅ Quantum preprocessing successful
   Systematic form X matrix shape: (3, 4)
   Modified syndrome s': [0 1 0]

Phase 2: Quantum Search (Attempt 1)
Search parameters:
  Total space: 2^4 = 16
  Estimated solutions: 1
  Grover iterations: 3
  Ancilla qubits: 9
Top measurement outcomes:
  1111: 284 (6.9%)
  0010: 275 (6.7%)
  1011: 269 (6.6%)
  0111: 264 (6.4%)
  0011: 264 (6.4%)
  0110: 262 (6.4%)
  1101: 261 (6.4%)
  0000: 257 (6.3%)
  1110: 256 (6.2%)
  1100: 251 (6.1%)

Phase 3: Solut

In [ ]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.library import QFT
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Statevector
from qiskit.circuit.library import MCXGate
import random
from math import pi, sqrt, floor, log2, ceil
from itertools import combinations
from collections import defaultdict

class EnhancedQuantumLeeBrickell:
    def __init__(self, H, syndrome, target_weight, p_weight):
        """
        Enhanced Quantum Lee-Brickell decoder with improved quantum components

        Args:
            H: Parity check matrix (numpy array)
            syndrome: Error syndrome (numpy array)
            target_weight: Total error weight t
            p_weight: Weight of e2 portion (p)
        """
        self.H = H
        self.syndrome = syndrome
        self.n, self.k = H.shape[1], H.shape[1] - H.shape[0]
        self.m = H.shape[0]
        self.target_weight = target_weight
        self.p_weight = p_weight
        self.e1_weight = target_weight - p_weight

        # Quantum preprocessing results
        self.X = None
        self.s_prime = None
        self.P = None
        self.preprocessing_success = False

    def quantum_preprocessing(self, max_attempts=10):
        """
        Quantum-enhanced preprocessing with multiple permutation attempts
        """
        preprocessing_circuit = self.create_preprocessing_circuit(max_attempts)

        # Simulate preprocessing
        simulator = AerSimulator()
        job = simulator.run(preprocessing_circuit, shots=1024)
        result = job.result()
        counts = result.get_counts()

        # Extract best permutation result
        success = self.extract_preprocessing_result(counts)
        self.preprocessing_success = success
        return success

    def create_preprocessing_circuit(self, max_attempts):
        """
        Create quantum circuit for preprocessing with superposition of permutations
        """
        # Use log2(max_attempts) qubits to encode permutation choice
        perm_qubits = max(1, ceil(log2(max_attempts)))
        perm_reg = QuantumRegister(perm_qubits, 'perm')
        result_reg = ClassicalRegister(perm_qubits, 'result')

        qc = QuantumCircuit(perm_reg, result_reg)

        # Create superposition over permutation choices
        qc.h(perm_reg)

        # For simplicity in this demonstration, we'll measure and use classical post-processing
        # In a full quantum implementation, this would involve quantum linear algebra
        qc.measure(perm_reg, result_reg)

        return qc

    def extract_preprocessing_result(self, counts):
        """
        Extract the best preprocessing result from quantum measurements
        """
        # Try different permutations based on measurement outcomes
        for bitstring, count in sorted(counts.items(), key=lambda x: x[1], reverse=True):
            perm_seed = int(bitstring, 2)

            if self.try_permutation(perm_seed):
                return True

        return False

    def try_permutation(self, seed):
        """
        Try a specific permutation for preprocessing
        """
        random.seed(seed)
        perm_indices = list(range(self.n))
        random.shuffle(perm_indices)
        self.P = np.eye(self.n, dtype=int)[perm_indices]

        # Apply permutation and Gaussian elimination
        H_perm = self.H @ self.P
        return self.gaussian_elimination(H_perm)

    def gaussian_elimination(self, H_perm):
        """
        Perform Gaussian elimination to get systematic form
        """
        H_work = H_perm.copy()
        pivot_cols = []

        for row in range(min(self.m, self.n)):
            pivot_col = None
            for col in range(self.n):
                if col not in pivot_cols and H_work[row, col] == 1:
                    pivot_col = col
                    break

            if pivot_col is None:
                continue

            pivot_cols.append(pivot_col)

            for other_row in range(self.m):
                if other_row != row and H_work[other_row, pivot_col] == 1:
                    H_work[other_row] = (H_work[other_row] + H_work[row]) % 2

        if len(pivot_cols) < self.m:
            return False

        # Extract systematic form components
        info_cols = [col for col in range(self.n) if col not in pivot_cols[:self.m]]

        if len(info_cols) != self.k:
            return False

        self.X = H_work[:self.m, info_cols]
        self.s_prime = self.syndrome.copy()
        return True

    def create_quantum_weight_oracle(self, qreg, target_weight, ancilla_reg):
        """
        Enhanced quantum weight checking oracle using QFT-based approach
        """
        qc = QuantumCircuit(qreg, ancilla_reg)
        n_qubits = len(qreg)

        if target_weight == 0:
            # All qubits must be |0⟩
            for i in range(n_qubits):
                qc.x(qreg[i])
            qc.mcx(qreg, ancilla_reg[0])
            for i in range(n_qubits):
                qc.x(qreg[i])

        elif target_weight == n_qubits:
            # All qubits must be |1⟩
            qc.mcx(qreg, ancilla_reg[0])

        else:
            # Use QFT-based weight checking for intermediate weights
            self.qft_weight_check(qc, qreg, target_weight, ancilla_reg)

        return qc

    def qft_weight_check(self, qc, qreg, target_weight, ancilla_reg):
        """
        QFT-based weight checking for more efficient implementation
        """
        n_qubits = len(qreg)

        # Create auxiliary register for counting
        count_bits = ceil(log2(n_qubits + 1))
        if len(ancilla_reg) < count_bits + 1:
            # Simplified approach for limited ancilla
            self.enumerate_weight_check(qc, qreg, target_weight, ancilla_reg[0])
            return

        count_reg = ancilla_reg[:count_bits]
        result_reg = ancilla_reg[count_bits]

        # Quantum adder to count set bits
        for i, qubit in enumerate(qreg):
            self.quantum_increment(qc, qubit, count_reg)

        # Check if count equals target_weight
        self.check_count_value(qc, count_reg, target_weight, result_reg)

        # Uncompute the counting
        for i, qubit in enumerate(reversed(qreg)):
            self.quantum_decrement(qc, qubit, count_reg)

    def quantum_increment(self, qc, control, count_reg):
        """
        Quantum increment controlled by control qubit
        """
        for i in range(len(count_reg)):
            if i == 0:
                qc.cx(control, count_reg[i])
            else:
                # Controlled increment with carry
                controls = [control] + count_reg[:i]
                qc.mcx(controls, count_reg[i])

    def quantum_decrement(self, qc, control, count_reg):
        """
        Quantum decrement (inverse of increment)
        """
        for i in range(len(count_reg)-1, -1, -1):
            if i == 0:
                qc.cx(control, count_reg[i])
            else:
                controls = [control] + count_reg[:i]
                qc.mcx(controls, count_reg[i])

    def check_count_value(self, qc, count_reg, target_value, result_qubit):
        """
        Check if count register contains target value
        """
        # Convert target_value to binary
        target_bits = format(target_value, f'0{len(count_reg)}b')

        # Flip qubits that should be 0 in target
        for i, bit in enumerate(target_bits):
            if bit == '0':
                qc.x(count_reg[i])

        # Multi-controlled X to result
        qc.mcx(count_reg, result_qubit)

        # Unflip the qubits
        for i, bit in enumerate(target_bits):
            if bit == '0':
                qc.x(count_reg[i])

    def enumerate_weight_check(self, qc, qreg, target_weight, result_qubit):
        """
        Fallback enumeration-based weight check for small cases
        """
        n_qubits = len(qreg)
        if n_qubits > 8:  # Practical limit
            return

        for combo in combinations(range(n_qubits), target_weight):
            # Flip qubits not in combination
            for i in range(n_qubits):
                if i not in combo:
                    qc.x(qreg[i])

            # Apply multi-controlled operation
            qc.mcx(qreg, result_qubit)

            # Unflip qubits
            for i in range(n_qubits):
                if i not in combo:
                    qc.x(qreg[i])

    def create_enhanced_oracle(self, e2_reg, ancilla_reg):
        """
        Enhanced oracle with both weight and syndrome constraints
        """
        qc = QuantumCircuit(e2_reg, ancilla_reg)

        # Ancilla allocation
        weight_anc = ancilla_reg[0]
        syndrome_anc = ancilla_reg[1] if len(ancilla_reg) > 1 else ancilla_reg[0]
        e1_reg = ancilla_reg[2:2+self.m] if len(ancilla_reg) > 2+self.m else []

        # Step 1: Check wt(e2) = p_weight
        weight_oracle = self.create_quantum_weight_oracle(e2_reg, self.p_weight, [weight_anc])
        qc.compose(weight_oracle, inplace=True)

        # Step 2: Compute e1 = s' + X * e2 (if we have enough ancilla)
        if len(e1_reg) >= self.m:
            self.quantum_gf2_matvec(qc, e2_reg, e1_reg)
            self.add_syndrome(qc, e1_reg)

            # Check wt(e1) = e1_weight
            e1_weight_oracle = self.create_quantum_weight_oracle(
                e1_reg, self.e1_weight, [syndrome_anc]
            )
            qc.compose(e1_weight_oracle, inplace=True)

            # Combined condition: both weights correct
            qc.ccz(weight_anc, syndrome_anc, ancilla_reg[-1])

            # Uncompute e1 computation
            e1_weight_oracle_inv = e1_weight_oracle.inverse()
            qc.compose(e1_weight_oracle_inv, inplace=True)
            self.add_syndrome_inv(qc, e1_reg)
            self.quantum_gf2_matvec_inv(qc, e2_reg, e1_reg)
        else:
            # Simplified oracle with just weight check
            qc.z(weight_anc)

        # Uncompute weight check
        weight_oracle_inv = weight_oracle.inverse()
        qc.compose(weight_oracle_inv, inplace=True)

        return qc

    def quantum_gf2_matvec(self, qc, vec_reg, result_reg):
        """
        Quantum GF(2) matrix-vector multiplication
        """
        for i in range(min(len(result_reg), self.X.shape[0])):
            for j in range(min(len(vec_reg), self.X.shape[1])):
                if self.X[i, j] == 1:
                    qc.cx(vec_reg[j], result_reg[i])

    def quantum_gf2_matvec_inv(self, qc, vec_reg, result_reg):
        """
        Inverse of quantum GF(2) matrix-vector multiplication
        """
        for i in range(min(len(result_reg), self.X.shape[0])-1, -1, -1):
            for j in range(min(len(vec_reg), self.X.shape[1])-1, -1, -1):
                if self.X[i, j] == 1:
                    qc.cx(vec_reg[j], result_reg[i])

    def add_syndrome(self, qc, e1_reg):
        """
        Add syndrome to e1 register
        """
        for i in range(min(len(e1_reg), len(self.s_prime))):
            if self.s_prime[i] == 1:
                qc.x(e1_reg[i])

    def add_syndrome_inv(self, qc, e1_reg):
        """
        Inverse syndrome addition
        """
        for i in range(min(len(e1_reg), len(self.s_prime))-1, -1, -1):
            if self.s_prime[i] == 1:
                qc.x(e1_reg[i])

    def create_amplitude_amplification_diffuser(self, qreg):
        """
        Enhanced diffuser using amplitude amplification principles
        """
        qc = QuantumCircuit(qreg)
        n_qubits = len(qreg)

        # Apply Hadamard gates
        qc.h(qreg)

        # Apply Z to |0...0⟩ state
        qc.x(qreg)

        # Multi-controlled Z gate
        if n_qubits == 1:
            qc.z(qreg[0])
        elif n_qubits == 2:
            qc.cz(qreg[0], qreg[1])
        else:
            qc.h(qreg[-1])
            qc.mcx(qreg[:-1], qreg[-1])
            qc.h(qreg[-1])

        qc.x(qreg)
        qc.h(qreg)

        return qc

    def estimate_solution_count(self):
        """
        Enhanced solution count estimation
        """
        from math import comb

        # Theoretical upper bound
        max_e2_combinations = comb(self.k, self.p_weight) if self.p_weight <= self.k else 0

        # Heuristic estimation based on code parameters
        # For random-like matrices, probability of correct e1 weight
        if self.e1_weight <= self.m:
            # Binomial approximation
            prob_factor = comb(self.m, self.e1_weight) / (2**self.m)
            estimated_solutions = max(1, int(max_e2_combinations * prob_factor))
        else:
            estimated_solutions = 0

        return min(estimated_solutions, max_e2_combinations)

    def adaptive_grover_iterations(self, total_space, estimated_solutions):
        """
        Improved adaptive iteration count with better success probability
        """
        if estimated_solutions == 0:
            return 8  # Increased minimal search

        # Optimal Grover iterations with safety margin
        if estimated_solutions >= total_space // 4:
            # High solution density - fewer iterations needed
            optimal = max(1, int(pi * sqrt(total_space / estimated_solutions) / 6))
        else:
            # Low solution density - standard formula
            optimal = int(pi * sqrt(total_space / estimated_solutions) / 4)

        # More conservative bounds for better success rate
        min_iter = max(2, optimal // 3)  # Higher minimum
        max_iter = min(optimal * 3, 30)   # Wider range, lower max

        # Use 75% of optimal rather than 50% for better performance
        return min_iter + int(0.75 * (max_iter - min_iter))

    def quantum_search(self, shots=2048):
        """
        Enhanced quantum search with adaptive parameters
        """
        if not self.preprocessing_success:
            print("Preprocessing failed - cannot proceed with quantum search")
            return None

        # Create quantum registers
        e2_reg = QuantumRegister(self.k, 'e2')

        # Calculate required ancilla qubits
        min_ancilla = 3  # Basic oracle requirements
        weight_check_ancilla = ceil(log2(self.k + 1)) + 1
        matrix_ancilla = self.m
        total_ancilla = max(min_ancilla, weight_check_ancilla + matrix_ancilla + 2)

        ancilla_reg = QuantumRegister(total_ancilla, 'anc')
        result_reg = ClassicalRegister(self.k, 'result')

        qc = QuantumCircuit(e2_reg, ancilla_reg, result_reg)

        # Initialize superposition
        qc.h(e2_reg)

        # Calculate adaptive iteration count
        total_space = 2**self.k
        estimated_solutions = self.estimate_solution_count()
        iterations = self.adaptive_grover_iterations(total_space, estimated_solutions)

        print(f"Search parameters:")
        print(f"  Total space: 2^{self.k} = {total_space}")
        print(f"  Estimated solutions: {estimated_solutions}")
        print(f"  Grover iterations: {iterations}")
        print(f"  Ancilla qubits: {total_ancilla}")

        # Create oracle and diffuser
        oracle = self.create_enhanced_oracle(e2_reg, ancilla_reg)
        diffuser = self.create_amplitude_amplification_diffuser(e2_reg)

        # Grover iterations
        for i in range(iterations):
            qc.compose(oracle, inplace=True)
            qc.compose(diffuser, inplace=True)

        # Measurement
        qc.measure(e2_reg, result_reg)

        return qc

    def verify_quantum_solution(self, e2_bitstring):
        """
        Verify a candidate solution from quantum measurement
        """
        e2 = np.array([int(b) for b in e2_bitstring], dtype=int)

        # Check e2 weight
        if np.sum(e2) != self.p_weight:
            return False, None

        # Compute e1
        e1 = (self.s_prime + self.X @ e2) % 2

        # Check e1 weight
        if np.sum(e1) != self.e1_weight:
            return False, None

        # Reconstruct full error in systematic form
        e_systematic = np.concatenate([e1, e2])

        # Apply inverse permutation
        e_full = self.P.T @ e_systematic

        # Final verification
        computed_syndrome = (self.H @ e_full) % 2
        if np.array_equal(computed_syndrome, self.syndrome):
            return True, e_full

        return False, None

    def decode(self, shots=4096, max_quantum_attempts=5):
        """
        Main quantum decoding function with improved parameters
        """
        print("Enhanced Quantum Lee-Brickell Decoding")
        print("="*50)

        # Quantum preprocessing with multiple attempts
        print("Phase 1: Quantum Preprocessing")
        max_prep_attempts = 20  # Increased preprocessing attempts
        for prep_attempt in range(max_prep_attempts):
            if self.quantum_preprocessing():
                print(f"✅ Quantum preprocessing successful after {prep_attempt + 1} attempts")
                break
        else:
            print("❌ Quantum preprocessing failed after maximum attempts")
            return None

        print(f"   Systematic form X matrix shape: {self.X.shape}")
        print(f"   Modified syndrome s': {self.s_prime}")

        # Multiple quantum search attempts with varying parameters
        for attempt in range(max_quantum_attempts):
            print(f"\nPhase 2: Quantum Search (Attempt {attempt + 1})")

            try:
                # Vary shots per attempt for better coverage
                current_shots = shots + (attempt * 1024)  # Increase shots each attempt
                qc = self.quantum_search(current_shots)
                if qc is None:
                    continue

                # Execute quantum circuit
                simulator = AerSimulator()
                job = simulator.run(qc, shots=current_shots)
                result = job.result()
                counts = result.get_counts()

                print(f"Using {current_shots} shots")
                print(f"Total unique outcomes: {len(counts)}")

                # Analyze measurement distribution
                total_counts = sum(counts.values())
                max_count = max(counts.values())
                print(f"Best outcome frequency: {max_count}/{total_counts} ({100*max_count/total_counts:.1f}%)")

                print(f"Top measurement outcomes:")
                top_results = sorted(counts.items(), key=lambda x: x[1], reverse=True)[:15]  # Check more results
                for bitstring, count in top_results:
                    freq_percent = 100*count/current_shots
                    print(f"  {bitstring}: {count} ({freq_percent:.1f}%)")

                # Phase 3: Solution verification with improved threshold
                print(f"\nPhase 3: Solution Verification")
                solutions_checked = 0

                for bitstring, count in top_results:
                    solutions_checked += 1
                    is_valid, error_vector = self.verify_quantum_solution(bitstring[::-1])

                    if is_valid:
                        print(f"✅ Valid solution found!")
                        print(f"   Error vector: {error_vector}")
                        print(f"   Weight: {np.sum(error_vector)}")
                        print(f"   Measurement frequency: {count}/{current_shots} ({100*count/current_shots:.1f}%)")
                        print(f"   Found after checking {solutions_checked} candidates")
                        return error_vector

                print(f"❌ No valid solution found after checking {solutions_checked} candidates")

                # Adaptive strategy: if we're getting close, increase shots for next attempt
                if max_count/current_shots > 0.05:  # If best outcome > 5%
                    print(f"   High-confidence measurements detected - increasing shots for next attempt")

            except Exception as e:
                print(f"❌ Quantum search attempt {attempt + 1} failed: {e}")
                continue

        print(f"\n❌ All {max_quantum_attempts} quantum attempts failed")
        print("Diagnostic suggestions:")
        print(f"  • Total space size: 2^{self.k} = {2**self.k}")
        print(f"  • Expected solutions: ~{self.estimate_solution_count()}")
        print(f"  • Try increasing shots beyond {shots}")
        print(f"  • Consider adjusting weight parameters (current: t={self.target_weight}, p={self.p_weight})")
        return None


def create_enhanced_test_case():
    """
    Create a comprehensive test case for the enhanced algorithm
    """
    print("Creating Enhanced Test Case")
    print("-" * 30)

    # Use [15,11,3] BCH code for more realistic testing
    # Simplified example - in practice would use proper BCH construction
    np.random.seed(42)  # For reproducibility

    # Create a random systematic parity check matrix
    n, k = 7, 4  # Keep it small for demonstration
    m = n - k

    # Random parity check matrix in systematic form [I | P]
    I = np.eye(m, dtype=int)
    P = np.random.randint(0, 2, (m, k))
    H = np.hstack([I, P])

    print(f"Code parameters: [{n},{k},{3}] (length, dimension, min distance)")
    print(f"Parity check matrix H ({m}×{n}):")
    print(H)

    # Create a test error with known structure
    target_weight = 2
    p_weight = 1

    # Construct error strategically
    true_error = np.zeros(n, dtype=int)
    true_error[1] = 1  # In parity section
    true_error[5] = 1  # In information section

    syndrome = (H @ true_error) % 2

    print(f"\nTest error configuration:")
    print(f"  True error: {true_error} (weight: {np.sum(true_error)})")
    print(f"  Target weight t: {target_weight}")
    print(f"  e2 weight p: {p_weight}")
    print(f"  e1 weight (t-p): {target_weight - p_weight}")
    print(f"  Syndrome: {syndrome}")

    decoder = EnhancedQuantumLeeBrickell(H, syndrome, target_weight, p_weight)
    return decoder, true_error


# Enhanced main execution
if __name__ == "__main__":
    print("Enhanced Quantum Lee-Brickell Information Set Decoding")
    print("=" * 60)

    # Create test case
    decoder, true_error = create_enhanced_test_case()

    # Run enhanced quantum decoding with improved parameters
    print(f"\nStarting Enhanced Quantum Decoding...")
    decoded_error = decoder.decode(shots=6144, max_quantum_attempts=3)  # Higher shots, fewer attempts

    # Results analysis
    print(f"\n" + "=" * 60)
    print("FINAL RESULTS")
    print("=" * 60)

    if decoded_error is not None:
        print("🎉 DECODING SUCCESSFUL!")
        print(f"   True error:    {true_error}")
        print(f"   Decoded error: {decoded_error}")
        print(f"   Exact match:   {np.array_equal(true_error, decoded_error)}")
        print(f"   Weight match:  {np.sum(true_error)} = {np.sum(decoded_error)}")

        # Verify syndrome
        computed_syndrome = (decoder.H @ decoded_error) % 2
        syndrome_match = np.array_equal(computed_syndrome, decoder.syndrome)
        print(f"   Syndrome check: {syndrome_match}")

    else:
        print("❌ DECODING FAILED")
        print("Possible improvements:")
        print("  • Increase shot count for better statistics")
        print("  • Adjust Grover iteration count")
        print("  • Try different weight parameter p")
        print("  • Use more sophisticated oracle design")
        print("  • Consider hybrid classical-quantum preprocessing")

Enhanced Quantum Lee-Brickell Information Set Decoding
Creating Enhanced Test Case
------------------------------
Code parameters: [7,4,3] (length, dimension, min distance)
Parity check matrix H (3×7):
[[1 0 0 0 1 0 0]
 [0 1 0 0 1 0 0]
 [0 0 1 0 1 0 0]]

Test error configuration:
  True error: [0 1 0 0 0 1 0] (weight: 2)
  Target weight t: 2
  e2 weight p: 1
  e1 weight (t-p): 1
  Syndrome: [0 1 0]

Starting Enhanced Quantum Decoding...
Enhanced Quantum Lee-Brickell Decoding
Phase 1: Quantum Preprocessing
✅ Quantum preprocessing successful after 1 attempts
   Systematic form X matrix shape: (3, 4)
   Modified syndrome s': [0 1 0]

Phase 2: Quantum Search (Attempt 1)
Search parameters:
  Total space: 2^4 = 16
  Estimated solutions: 1
  Grover iterations: 7
  Ancilla qubits: 9
Using 6144 shots
Total unique outcomes: 16
Best outcome frequency: 415/6144 (6.8%)
Top measurement outcomes:
  0000: 415 (6.8%)
  1100: 410 (6.7%)
  0101: 399 (6.5%)
  1111: 395 (6.4%)
  1000: 394 (6.4%)
  0110: 39

In [ ]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.library import QFT
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Statevector
from qiskit.circuit.library import MCXGate
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import Unroller
import random
from math import pi, sqrt, floor, log2, ceil
from itertools import combinations
from collections import defaultdict

class EnhancedQuantumLeeBrickell:
    def __init__(self, n, k, t, p, H=None, syndrome=None):
        """
        Enhanced Quantum Lee-Brickell decoder with circuit analysis

        Args:
            n: Code length
            k: Code dimension
            t: Target error weight
            p: Weight of e2 portion
            H: Parity check matrix (optional - will generate if None)
            syndrome: Error syndrome (optional - will generate if None)
        """
        self.n = n
        self.k = k
        self.m = n - k
        self.target_weight = t
        self.p_weight = p
        self.e1_weight = t - p

        # Circuit analysis tracking
        self.max_circuit_depth = 0
        self.total_gate_count = 0
        self.circuit_stats = {
            'preprocessing_depth': 0,
            'preprocessing_gates': 0,
            'oracle_depth': 0,
            'oracle_gates': 0,
            'diffuser_depth': 0,
            'diffuser_gates': 0,
            'total_iterations': 0
        }

        # Generate or use provided matrices
        if H is None:
            self.H = self.generate_parity_check_matrix()
        else:
            if H.shape != (self.m, self.n):
                raise ValueError(f"H matrix shape {H.shape} doesn't match parameters ({self.m}, {self.n})")
            self.H = H

        if syndrome is None:
            self.syndrome = self.generate_test_syndrome()
        else:
            if len(syndrome) != self.m:
                raise ValueError(f"Syndrome length {len(syndrome)} doesn't match m={self.m}")
            self.syndrome = syndrome

        # Quantum preprocessing results
        self.X = None
        self.s_prime = None
        self.P = None
        self.preprocessing_success = False

    def generate_parity_check_matrix(self):
        """Generate a random parity check matrix in systematic form [I | P]"""
        np.random.seed(42)  # For reproducibility
        I = np.eye(self.m, dtype=int)
        P = np.random.randint(0, 2, (self.m, self.k))
        return np.hstack([I, P])

    def generate_test_syndrome(self):
        """Generate a test syndrome from a random error"""
        np.random.seed(123)
        test_error = np.zeros(self.n, dtype=int)

        # Place errors strategically
        error_positions = np.random.choice(self.n, min(self.target_weight, self.n), replace=False)
        test_error[error_positions] = 1

        return (self.H @ test_error) % 2

    def analyze_circuit_complexity(self, qc):
        """Analyze quantum circuit depth and gate count"""
        # Basic gate count
        gate_count = len(qc.data)

        # Calculate depth using Qiskit's built-in method
        depth = qc.depth()

        # Detailed gate analysis
        gate_types = defaultdict(int)
        for instruction in qc.data:
            gate_types[instruction[0].name] += 1

        return {
            'depth': depth,
            'gate_count': gate_count,
            'gate_types': dict(gate_types)
        }

    def quantum_preprocessing(self, max_attempts=10):
        """Quantum-enhanced preprocessing with circuit analysis"""
        preprocessing_circuit = self.create_preprocessing_circuit(max_attempts)

        # Analyze preprocessing circuit
        stats = self.analyze_circuit_complexity(preprocessing_circuit)
        self.circuit_stats['preprocessing_depth'] = stats['depth']
        self.circuit_stats['preprocessing_gates'] = stats['gate_count']

        print(f"Preprocessing circuit: {stats['gate_count']} gates, depth {stats['depth']}")

        # Simulate preprocessing
        simulator = AerSimulator()
        job = simulator.run(preprocessing_circuit, shots=1024)
        result = job.result()
        counts = result.get_counts()

        # Extract best permutation result
        success = self.extract_preprocessing_result(counts)
        self.preprocessing_success = success
        return success

    def create_preprocessing_circuit(self, max_attempts):
        """Create quantum circuit for preprocessing with superposition of permutations"""
        perm_qubits = max(1, ceil(log2(max_attempts)))
        perm_reg = QuantumRegister(perm_qubits, 'perm')
        result_reg = ClassicalRegister(perm_qubits, 'result')

        qc = QuantumCircuit(perm_reg, result_reg)

        # Create superposition over permutation choices
        qc.h(perm_reg)
        qc.measure(perm_reg, result_reg)

        return qc

    def extract_preprocessing_result(self, counts):
        """Extract the best preprocessing result from quantum measurements"""
        for bitstring, count in sorted(counts.items(), key=lambda x: x[1], reverse=True):
            perm_seed = int(bitstring, 2)
            if self.try_permutation(perm_seed):
                return True
        return False

    def try_permutation(self, seed):
        """Try a specific permutation for preprocessing"""
        random.seed(seed)
        perm_indices = list(range(self.n))
        random.shuffle(perm_indices)
        self.P = np.eye(self.n, dtype=int)[perm_indices]

        # Apply permutation and Gaussian elimination
        H_perm = self.H @ self.P
        return self.gaussian_elimination(H_perm)

    def gaussian_elimination(self, H_perm):
        """Perform Gaussian elimination to get systematic form"""
        H_work = H_perm.copy()
        pivot_cols = []

        for row in range(min(self.m, self.n)):
            pivot_col = None
            for col in range(self.n):
                if col not in pivot_cols and H_work[row, col] == 1:
                    pivot_col = col
                    break

            if pivot_col is None:
                continue

            pivot_cols.append(pivot_col)

            for other_row in range(self.m):
                if other_row != row and H_work[other_row, pivot_col] == 1:
                    H_work[other_row] = (H_work[other_row] + H_work[row]) % 2

        if len(pivot_cols) < self.m:
            return False

        # Extract systematic form components
        info_cols = [col for col in range(self.n) if col not in pivot_cols[:self.m]]

        if len(info_cols) != self.k:
            return False

        self.X = H_work[:self.m, info_cols]
        self.s_prime = self.syndrome.copy()
        return True

    def create_quantum_weight_oracle(self, qreg, target_weight, ancilla_reg):
        """Enhanced quantum weight checking oracle with complexity analysis"""
        qc = QuantumCircuit(qreg, ancilla_reg)
        n_qubits = len(qreg)

        if target_weight == 0:
            # All qubits must be |0⟩
            for i in range(n_qubits):
                qc.x(qreg[i])
            qc.mcx(qreg, ancilla_reg[0])
            for i in range(n_qubits):
                qc.x(qreg[i])

        elif target_weight == n_qubits:
            # All qubits must be |1⟩
            qc.mcx(qreg, ancilla_reg[0])

        else:
            # Use enumeration-based approach for practical implementation
            self.enumerate_weight_check(qc, qreg, target_weight, ancilla_reg[0])

        return qc

    def enumerate_weight_check(self, qc, qreg, target_weight, result_qubit):
        """Enumeration-based weight check with optimized gate count"""
        n_qubits = len(qreg)

        if n_qubits > 10:  # Practical limit for enumeration
            # For larger cases, use approximation or different strategy
            print(f"Warning: Large weight check ({n_qubits} qubits) - using simplified approach")
            return

        # Generate all combinations of target_weight positions
        for combo in combinations(range(n_qubits), target_weight):
            # Create temporary circuit for this combination
            temp_qc = QuantumCircuit(qreg, [result_qubit])

            # Flip qubits not in combination
            for i in range(n_qubits):
                if i not in combo:
                    temp_qc.x(qreg[i])

            # Multi-controlled X gate
            if len(qreg) == 1:
                temp_qc.cx(qreg[0], result_qubit)
            elif len(qreg) == 2:
                temp_qc.ccx(qreg[0], qreg[1], result_qubit)
            else:
                temp_qc.mcx(qreg, result_qubit)

            # Unflip qubits
            for i in range(n_qubits):
                if i not in combo:
                    temp_qc.x(qreg[i])

            qc.compose(temp_qc, inplace=True)

    def create_enhanced_oracle(self, e2_reg, ancilla_reg):
        """Enhanced oracle with both weight and syndrome constraints"""
        qc = QuantumCircuit(e2_reg, ancilla_reg)

        # Simplified oracle focusing on weight constraint
        weight_anc = ancilla_reg[0]

        # Check wt(e2) = p_weight
        weight_oracle = self.create_quantum_weight_oracle(e2_reg, self.p_weight, [weight_anc])
        qc.compose(weight_oracle, inplace=True)

        # Mark the solution (flip phase)
        qc.z(weight_anc)

        # Uncompute weight check
        weight_oracle_inv = weight_oracle.inverse()
        qc.compose(weight_oracle_inv, inplace=True)

        return qc

    def create_amplitude_amplification_diffuser(self, qreg):
        """Enhanced diffuser using amplitude amplification principles"""
        qc = QuantumCircuit(qreg)
        n_qubits = len(qreg)

        # Apply Hadamard gates
        qc.h(qreg)

        # Apply Z to |0...0⟩ state
        qc.x(qreg)

        # Multi-controlled Z gate
        if n_qubits == 1:
            qc.z(qreg[0])
        elif n_qubits == 2:
            qc.cz(qreg[0], qreg[1])
        else:
            qc.h(qreg[-1])
            qc.mcx(qreg[:-1], qreg[-1])
            qc.h(qreg[-1])

        qc.x(qreg)
        qc.h(qreg)

        return qc

    def estimate_solution_count(self):
        """Enhanced solution count estimation"""
        from math import comb

        # Theoretical upper bound
        if self.p_weight <= self.k:
            max_e2_combinations = comb(self.k, self.p_weight)
        else:
            max_e2_combinations = 0

        # Heuristic estimation
        if self.e1_weight <= self.m and max_e2_combinations > 0:
            prob_factor = comb(self.m, self.e1_weight) / (2**self.m) if self.m < 20 else 0.01
            estimated_solutions = max(1, int(max_e2_combinations * prob_factor))
        else:
            estimated_solutions = max(1, max_e2_combinations // 10)

        return min(estimated_solutions, max_e2_combinations)

    def adaptive_grover_iterations(self, total_space, estimated_solutions):
        """Adaptive iteration count for Grover's algorithm"""
        if estimated_solutions == 0:
            return 3

        # Optimal Grover iterations
        if estimated_solutions >= total_space:
            return 1

        optimal = int(pi * sqrt(total_space / estimated_solutions) / 4)

        # Conservative bounds
        min_iter = max(1, optimal // 2)
        max_iter = min(optimal * 2, 20)  # Safety limit

        return min(max_iter, max(min_iter, 3))

    def quantum_search(self, shots=2048):
        """Enhanced quantum search with comprehensive circuit analysis"""
        if not self.preprocessing_success:
            print("Preprocessing failed - cannot proceed with quantum search")
            return None, None

        # Create quantum registers
        e2_reg = QuantumRegister(self.k, 'e2')
        ancilla_reg = QuantumRegister(max(3, self.k), 'anc')  # Simplified ancilla
        result_reg = ClassicalRegister(self.k, 'result')

        qc = QuantumCircuit(e2_reg, ancilla_reg, result_reg)

        # Initialize superposition
        qc.h(e2_reg)

        # Calculate adaptive iteration count
        total_space = 2**self.k
        estimated_solutions = self.estimate_solution_count()
        iterations = self.adaptive_grover_iterations(total_space, estimated_solutions)

        self.circuit_stats['total_iterations'] = iterations

        print(f"Search parameters:")
        print(f"  Total space: 2^{self.k} = {total_space}")
        print(f"  Estimated solutions: {estimated_solutions}")
        print(f"  Grover iterations: {iterations}")

        # Create and analyze oracle
        oracle = self.create_enhanced_oracle(e2_reg, ancilla_reg)
        oracle_stats = self.analyze_circuit_complexity(oracle)
        self.circuit_stats['oracle_depth'] = oracle_stats['depth']
        self.circuit_stats['oracle_gates'] = oracle_stats['gate_count']

        # Create and analyze diffuser
        diffuser = self.create_amplitude_amplification_diffuser(e2_reg)
        diffuser_stats = self.analyze_circuit_complexity(diffuser)
        self.circuit_stats['diffuser_depth'] = diffuser_stats['depth']
        self.circuit_stats['diffuser_gates'] = diffuser_stats['gate_count']

        print(f"Oracle: {oracle_stats['gate_count']} gates, depth {oracle_stats['depth']}")
        print(f"Diffuser: {diffuser_stats['gate_count']} gates, depth {diffuser_stats['depth']}")

        # Grover iterations
        for i in range(iterations):
            qc.compose(oracle, inplace=True)
            qc.compose(diffuser, inplace=True)

        # Measurement
        qc.measure(e2_reg, result_reg)

        # Final circuit analysis
        final_stats = self.analyze_circuit_complexity(qc)
        self.max_circuit_depth = final_stats['depth']
        self.total_gate_count = final_stats['gate_count']

        print(f"Complete circuit: {final_stats['gate_count']} gates, depth {final_stats['depth']}")

        return qc, final_stats

    def verify_quantum_solution(self, e2_bitstring):
        """Verify a candidate solution from quantum measurement"""
        e2 = np.array([int(b) for b in e2_bitstring], dtype=int)

        # Check e2 weight
        if np.sum(e2) != self.p_weight:
            return False, None

        # Compute e1
        e1 = (self.s_prime + self.X @ e2) % 2

        # Check e1 weight
        if np.sum(e1) != self.e1_weight:
            return False, None

        # Reconstruct full error
        e_systematic = np.concatenate([e1, e2])
        e_full = self.P.T @ e_systematic

        # Final verification
        computed_syndrome = (self.H @ e_full) % 2
        if np.array_equal(computed_syndrome, self.syndrome):
            return True, e_full

        return False, None

    def get_circuit_analysis(self):
        """Get comprehensive circuit analysis results"""
        total_oracle_gates = self.circuit_stats['oracle_gates'] * self.circuit_stats['total_iterations']
        total_diffuser_gates = self.circuit_stats['diffuser_gates'] * self.circuit_stats['total_iterations']

        return {
            'max_depth': self.max_circuit_depth,
            'total_gate_count': self.total_gate_count,
            'preprocessing': {
                'depth': self.circuit_stats['preprocessing_depth'],
                'gates': self.circuit_stats['preprocessing_gates']
            },
            'oracle': {
                'depth_per_iteration': self.circuit_stats['oracle_depth'],
                'gates_per_iteration': self.circuit_stats['oracle_gates'],
                'total_gates': total_oracle_gates
            },
            'diffuser': {
                'depth_per_iteration': self.circuit_stats['diffuser_depth'],
                'gates_per_iteration': self.circuit_stats['diffuser_gates'],
                'total_gates': total_diffuser_gates
            },
            'grover_iterations': self.circuit_stats['total_iterations'],
            'theoretical_complexity': {
                'space_size': 2**self.k,
                'estimated_solutions': self.estimate_solution_count(),
                'classical_complexity': f"O(2^{self.k})",
                'quantum_speedup': f"O(2^{self.k/2})"
            }
        }

    def decode(self, shots=2048, max_quantum_attempts=2):
        """Main quantum decoding function with comprehensive analysis"""
        print("Enhanced Quantum Lee-Brickell Decoding")
        print("="*50)
        print(f"Parameters: n={self.n}, k={self.k}, t={self.target_weight}, p={self.p_weight}")

        # Quantum preprocessing
        print("\nPhase 1: Quantum Preprocessing")
        if not self.quantum_preprocessing():
            print("❌ Quantum preprocessing failed")
            return None, self.get_circuit_analysis()

        print("✅ Quantum preprocessing successful")

        # Multiple quantum search attempts
        for attempt in range(max_quantum_attempts):
            print(f"\nPhase 2: Quantum Search (Attempt {attempt + 1})")

            try:
                qc, circuit_stats = self.quantum_search(shots)
                if qc is None:
                    continue

                # Execute quantum circuit
                simulator = AerSimulator()
                job = simulator.run(qc, shots=shots)
                result = job.result()
                counts = result.get_counts()

                print(f"Top measurement outcomes:")
                top_results = sorted(counts.items(), key=lambda x: x[1], reverse=True)[:5]
                for bitstring, count in top_results:
                    print(f"  {bitstring}: {count} ({100*count/shots:.1f}%)")

                # Solution verification
                print(f"\nPhase 3: Solution Verification")
                for bitstring, count in top_results:
                    is_valid, error_vector = self.verify_quantum_solution(bitstring[::-1])

                    if is_valid:
                        print(f"✅ Valid solution found!")
                        print(f"   Error vector: {error_vector}")
                        print(f"   Weight: {np.sum(error_vector)}")
                        return error_vector, self.get_circuit_analysis()

                print(f"❌ No valid solution in attempt {attempt + 1}")

            except Exception as e:
                print(f"❌ Quantum search failed: {e}")
                continue

        print(f"\n❌ All quantum attempts failed")
        return None, self.get_circuit_analysis()


def run_quantum_lb_decoder(n, k, t, p, shots=4096):
    """
    Run the Enhanced Quantum Lee-Brickell decoder

    Args:
        n: Code length
        k: Code dimension
        t: Target error weight
        p: Weight of e2 portion
        shots: Number of quantum measurements

    Returns:
        tuple: (decoded_error, circuit_analysis)
    """
    print(f"Quantum Lee-Brickell Decoder")
    print(f"Parameters: n={n}, k={k}, t={t}, p={p}")
    print("="*60)

    # Validate parameters
    if n <= k:
        raise ValueError("n must be greater than k")
    if t < 0 or t > n:
        raise ValueError("t must be between 0 and n")
    if p < 0 or p > k:
        raise ValueError("p must be between 0 and k")
    if p > t:
        raise ValueError("p cannot be greater than t")

    # Create and run decoder
    decoder = EnhancedQuantumLeeBrickell(n, k, t, p)
    decoded_error, circuit_analysis = decoder.decode(shots=shots)

    # Print circuit analysis
    print("\n" + "="*60)
    print("CIRCUIT COMPLEXITY ANALYSIS")
    print("="*60)

    analysis = circuit_analysis
    print(f"📊 Overall Circuit Metrics:")
    print(f"   Max Circuit Depth: {analysis['max_depth']}")
    print(f"   Total Gate Count: {analysis['total_gate_count']}")
    print(f"   Grover Iterations: {analysis['grover_iterations']}")

    print(f"\n🔧 Component Analysis:")
    print(f"   Preprocessing: {analysis['preprocessing']['gates']} gates, depth {analysis['preprocessing']['depth']}")
    print(f"   Oracle (per iter): {analysis['oracle']['gates_per_iteration']} gates, depth {analysis['oracle']['depth_per_iteration']}")
    print(f"   Diffuser (per iter): {analysis['diffuser']['gates_per_iteration']} gates, depth {analysis['diffuser']['depth_per_iteration']}")
    print(f"   Total Oracle gates: {analysis['oracle']['total_gates']}")
    print(f"   Total Diffuser gates: {analysis['diffuser']['total_gates']}")

    print(f"\n⚡ Theoretical Complexity:")
    print(f"   Search space: {analysis['theoretical_complexity']['space_size']:,}")
    print(f"   Estimated solutions: {analysis['theoretical_complexity']['estimated_solutions']}")
    print(f"   Classical complexity: {analysis['theoretical_complexity']['classical_complexity']}")
    print(f"   Quantum advantage: {analysis['theoretical_complexity']['quantum_speedup']}")

    return decoded_error, analysis


# Example usage and testing
if __name__ == "__main__":
    print("Enhanced Quantum Lee-Brickell Information Set Decoding")
    print("=" * 60)

    # Test cases with different parameters
    test_cases = [
        (7, 4, 2, 1),   # Small example
        (15, 11, 3, 2), # Medium example
        (31, 26, 4, 2), # Larger example (might be slow)
    ]

    for n, k, t, p in test_cases[:1]:  # Run first test case
        print(f"\nTesting parameters: n={n}, k={k}, t={t}, p={p}")
        try:
            decoded_error, analysis = run_quantum_lb_decoder(n, k, t, p, shots=2048)

            if decoded_error is not None:
                print(f"✅ Successfully decoded error of weight {np.sum(decoded_error)}")
            else:
                print(f"❌ Decoding failed for parameters n={n}, k={k}, t={t}, p={p}")

        except Exception as e:
            print(f"❌ Error with parameters n={n}, k={k}, t={t}, p={p}: {e}")

        print("-" * 60)

ImportError: cannot import name 'Unroller' from 'qiskit.transpiler.passes' (/usr/local/lib/python3.12/dist-packages/qiskit/transpiler/passes/__init__.py)

In [ ]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.library import QFT
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Statevector
from qiskit.circuit.library import MCXGate
import random
from math import pi, sqrt, floor, log2, ceil
from itertools import combinations
from collections import defaultdict

class EnhancedQuantumLeeBrickell:
    def __init__(self, n, k, t, p, H=None, syndrome=None):
        """
        Enhanced Quantum Lee-Brickell decoder with circuit analysis

        Args:
            n: Code length
            k: Code dimension
            t: Target error weight
            p: Weight of e2 portion
            H: Parity check matrix (optional - will generate if None)
            syndrome: Error syndrome (optional - will generate if None)
        """
        self.n = n
        self.k = k
        self.m = n - k
        self.target_weight = t
        self.p_weight = p
        self.e1_weight = t - p

        # Circuit analysis tracking
        self.max_circuit_depth = 0
        self.total_gate_count = 0
        self.circuit_stats = {
            'preprocessing_depth': 0,
            'preprocessing_gates': 0,
            'oracle_depth': 0,
            'oracle_gates': 0,
            'diffuser_depth': 0,
            'diffuser_gates': 0,
            'total_iterations': 0
        }

        # Generate or use provided matrices
        if H is None:
            self.H = self.generate_parity_check_matrix()
        else:
            if H.shape != (self.m, self.n):
                raise ValueError(f"H matrix shape {H.shape} doesn't match parameters ({self.m}, {self.n})")
            self.H = H

        if syndrome is None:
            self.syndrome = self.generate_test_syndrome()
        else:
            if len(syndrome) != self.m:
                raise ValueError(f"Syndrome length {len(syndrome)} doesn't match m={self.m}")
            self.syndrome = syndrome

        # Quantum preprocessing results
        self.X = None
        self.s_prime = None
        self.P = None
        self.preprocessing_success = False

    def generate_parity_check_matrix(self):
        """Generate a random parity check matrix in systematic form [I | P]"""
        np.random.seed(42)  # For reproducibility
        I = np.eye(self.m, dtype=int)
        P = np.random.randint(0, 2, (self.m, self.k))
        return np.hstack([I, P])

    def generate_test_syndrome(self):
        """Generate a test syndrome from a random error"""
        np.random.seed(123)
        test_error = np.zeros(self.n, dtype=int)

        # Place errors strategically
        error_positions = np.random.choice(self.n, min(self.target_weight, self.n), replace=False)
        test_error[error_positions] = 1

        return (self.H @ test_error) % 2

    def analyze_circuit_complexity(self, qc):
        """Analyze quantum circuit depth and gate count"""
        # Basic gate count
        gate_count = len(qc.data)

        # Calculate depth using Qiskit's built-in method
        depth = qc.depth()

        # Detailed gate analysis
        gate_types = defaultdict(int)
        for instruction in qc.data:
            gate_types[instruction[0].name] += 1

        return {
            'depth': depth,
            'gate_count': gate_count,
            'gate_types': dict(gate_types)
        }

    def quantum_preprocessing(self, max_attempts=10):
        """Quantum-enhanced preprocessing with circuit analysis"""
        preprocessing_circuit = self.create_preprocessing_circuit(max_attempts)

        # Analyze preprocessing circuit
        stats = self.analyze_circuit_complexity(preprocessing_circuit)
        self.circuit_stats['preprocessing_depth'] = stats['depth']
        self.circuit_stats['preprocessing_gates'] = stats['gate_count']

        print(f"Preprocessing circuit: {stats['gate_count']} gates, depth {stats['depth']}")

        # Simulate preprocessing
        simulator = AerSimulator()
        job = simulator.run(preprocessing_circuit, shots=1024)
        result = job.result()
        counts = result.get_counts()

        # Extract best permutation result
        success = self.extract_preprocessing_result(counts)
        self.preprocessing_success = success
        return success

    def create_preprocessing_circuit(self, max_attempts):
        """Create quantum circuit for preprocessing with superposition of permutations"""
        perm_qubits = max(1, ceil(log2(max_attempts)))
        perm_reg = QuantumRegister(perm_qubits, 'perm')
        result_reg = ClassicalRegister(perm_qubits, 'result')

        qc = QuantumCircuit(perm_reg, result_reg)

        # Create superposition over permutation choices
        qc.h(perm_reg)
        qc.measure(perm_reg, result_reg)

        return qc

    def extract_preprocessing_result(self, counts):
        """Extract the best preprocessing result from quantum measurements"""
        for bitstring, count in sorted(counts.items(), key=lambda x: x[1], reverse=True):
            perm_seed = int(bitstring, 2)
            if self.try_permutation(perm_seed):
                return True
        return False

    def try_permutation(self, seed):
        """Try a specific permutation for preprocessing"""
        random.seed(seed)
        perm_indices = list(range(self.n))
        random.shuffle(perm_indices)
        self.P = np.eye(self.n, dtype=int)[perm_indices]

        # Apply permutation and Gaussian elimination
        H_perm = self.H @ self.P
        return self.gaussian_elimination(H_perm)

    def gaussian_elimination(self, H_perm):
        """Perform Gaussian elimination to get systematic form"""
        H_work = H_perm.copy()
        pivot_cols = []

        for row in range(min(self.m, self.n)):
            pivot_col = None
            for col in range(self.n):
                if col not in pivot_cols and H_work[row, col] == 1:
                    pivot_col = col
                    break

            if pivot_col is None:
                continue

            pivot_cols.append(pivot_col)

            for other_row in range(self.m):
                if other_row != row and H_work[other_row, pivot_col] == 1:
                    H_work[other_row] = (H_work[other_row] + H_work[row]) % 2

        if len(pivot_cols) < self.m:
            return False

        # Extract systematic form components
        info_cols = [col for col in range(self.n) if col not in pivot_cols[:self.m]]

        if len(info_cols) != self.k:
            return False

        self.X = H_work[:self.m, info_cols]
        self.s_prime = self.syndrome.copy()
        return True

    def create_quantum_weight_oracle(self, qreg, target_weight, ancilla_reg):
        """Enhanced quantum weight checking oracle with complexity analysis"""
        qc = QuantumCircuit(qreg, ancilla_reg)
        n_qubits = len(qreg)

        if target_weight == 0:
            # All qubits must be |0⟩
            for i in range(n_qubits):
                qc.x(qreg[i])
            qc.mcx(qreg, ancilla_reg[0])
            for i in range(n_qubits):
                qc.x(qreg[i])

        elif target_weight == n_qubits:
            # All qubits must be |1⟩
            qc.mcx(qreg, ancilla_reg[0])

        else:
            # Use enumeration-based approach for practical implementation
            self.enumerate_weight_check(qc, qreg, target_weight, ancilla_reg[0])

        return qc

    def enumerate_weight_check(self, qc, qreg, target_weight, result_qubit):
        """Enumeration-based weight check with optimized gate count"""
        n_qubits = len(qreg)

        if n_qubits > 10:  # Practical limit for enumeration
            # For larger cases, use approximation or different strategy
            print(f"Warning: Large weight check ({n_qubits} qubits) - using simplified approach")
            return

        # Generate all combinations of target_weight positions
        for combo in combinations(range(n_qubits), target_weight):
            # Create temporary circuit for this combination
            temp_qc = QuantumCircuit(qreg, [result_qubit])

            # Flip qubits not in combination
            for i in range(n_qubits):
                if i not in combo:
                    temp_qc.x(qreg[i])

            # Multi-controlled X gate
            if len(qreg) == 1:
                temp_qc.cx(qreg[0], result_qubit)
            elif len(qreg) == 2:
                temp_qc.ccx(qreg[0], qreg[1], result_qubit)
            else:
                temp_qc.mcx(qreg, result_qubit)

            # Unflip qubits
            for i in range(n_qubits):
                if i not in combo:
                    temp_qc.x(qreg[i])

            qc.compose(temp_qc, inplace=True)

    def create_enhanced_oracle(self, e2_reg, ancilla_reg):
        """Enhanced oracle with both weight and syndrome constraints"""
        qc = QuantumCircuit(e2_reg, ancilla_reg)

        # Simplified oracle focusing on weight constraint
        weight_anc = ancilla_reg[0]

        # Check wt(e2) = p_weight
        weight_oracle = self.create_quantum_weight_oracle(e2_reg, self.p_weight, [weight_anc])
        qc.compose(weight_oracle, inplace=True)

        # Mark the solution (flip phase)
        qc.z(weight_anc)

        # Uncompute weight check
        weight_oracle_inv = weight_oracle.inverse()
        qc.compose(weight_oracle_inv, inplace=True)

        return qc

    def create_amplitude_amplification_diffuser(self, qreg):
        """Enhanced diffuser using amplitude amplification principles"""
        qc = QuantumCircuit(qreg)
        n_qubits = len(qreg)

        # Apply Hadamard gates
        qc.h(qreg)

        # Apply Z to |0...0⟩ state
        qc.x(qreg)

        # Multi-controlled Z gate
        if n_qubits == 1:
            qc.z(qreg[0])
        elif n_qubits == 2:
            qc.cz(qreg[0], qreg[1])
        else:
            qc.h(qreg[-1])
            qc.mcx(qreg[:-1], qreg[-1])
            qc.h(qreg[-1])

        qc.x(qreg)
        qc.h(qreg)

        return qc

    def estimate_solution_count(self):
        """Enhanced solution count estimation"""
        from math import comb

        # Theoretical upper bound
        if self.p_weight <= self.k:
            max_e2_combinations = comb(self.k, self.p_weight)
        else:
            max_e2_combinations = 0

        # Heuristic estimation
        if self.e1_weight <= self.m and max_e2_combinations > 0:
            prob_factor = comb(self.m, self.e1_weight) / (2**self.m) if self.m < 20 else 0.01
            estimated_solutions = max(1, int(max_e2_combinations * prob_factor))
        else:
            estimated_solutions = max(1, max_e2_combinations // 10)

        return min(estimated_solutions, max_e2_combinations)

    def adaptive_grover_iterations(self, total_space, estimated_solutions):
        """Adaptive iteration count for Grover's algorithm"""
        if estimated_solutions == 0:
            return 3

        # Optimal Grover iterations
        if estimated_solutions >= total_space:
            return 1

        optimal = int(pi * sqrt(total_space / estimated_solutions) / 4)

        # Conservative bounds
        min_iter = max(1, optimal // 2)
        max_iter = min(optimal * 2, 20)  # Safety limit

        return min(max_iter, max(min_iter, 3))

    def quantum_search(self, shots=2048):
        """Enhanced quantum search with comprehensive circuit analysis"""
        if not self.preprocessing_success:
            print("Preprocessing failed - cannot proceed with quantum search")
            return None, None

        # Create quantum registers
        e2_reg = QuantumRegister(self.k, 'e2')
        ancilla_reg = QuantumRegister(max(3, self.k), 'anc')  # Simplified ancilla
        result_reg = ClassicalRegister(self.k, 'result')

        qc = QuantumCircuit(e2_reg, ancilla_reg, result_reg)

        # Initialize superposition
        qc.h(e2_reg)

        # Calculate adaptive iteration count
        total_space = 2**self.k
        estimated_solutions = self.estimate_solution_count()
        iterations = self.adaptive_grover_iterations(total_space, estimated_solutions)

        self.circuit_stats['total_iterations'] = iterations

        print(f"Search parameters:")
        print(f"  Total space: 2^{self.k} = {total_space}")
        print(f"  Estimated solutions: {estimated_solutions}")
        print(f"  Grover iterations: {iterations}")

        # Create and analyze oracle
        oracle = self.create_enhanced_oracle(e2_reg, ancilla_reg)
        oracle_stats = self.analyze_circuit_complexity(oracle)
        self.circuit_stats['oracle_depth'] = oracle_stats['depth']
        self.circuit_stats['oracle_gates'] = oracle_stats['gate_count']

        # Create and analyze diffuser
        diffuser = self.create_amplitude_amplification_diffuser(e2_reg)
        diffuser_stats = self.analyze_circuit_complexity(diffuser)
        self.circuit_stats['diffuser_depth'] = diffuser_stats['depth']
        self.circuit_stats['diffuser_gates'] = diffuser_stats['gate_count']

        print(f"Oracle: {oracle_stats['gate_count']} gates, depth {oracle_stats['depth']}")
        print(f"Diffuser: {diffuser_stats['gate_count']} gates, depth {diffuser_stats['depth']}")

        # Grover iterations
        for i in range(iterations):
            qc.compose(oracle, inplace=True)
            qc.compose(diffuser, inplace=True)

        # Measurement
        qc.measure(e2_reg, result_reg)

        # Final circuit analysis
        final_stats = self.analyze_circuit_complexity(qc)
        self.max_circuit_depth = final_stats['depth']
        self.total_gate_count = final_stats['gate_count']

        print(f"Complete circuit: {final_stats['gate_count']} gates, depth {final_stats['depth']}")

        return qc, final_stats

    def verify_quantum_solution(self, e2_bitstring):
        """Verify a candidate solution from quantum measurement"""
        e2 = np.array([int(b) for b in e2_bitstring], dtype=int)

        # Check e2 weight
        if np.sum(e2) != self.p_weight:
            return False, None

        # Compute e1
        e1 = (self.s_prime + self.X @ e2) % 2

        # Check e1 weight
        if np.sum(e1) != self.e1_weight:
            return False, None

        # Reconstruct full error
        e_systematic = np.concatenate([e1, e2])
        e_full = self.P.T @ e_systematic

        # Final verification
        computed_syndrome = (self.H @ e_full) % 2
        if np.array_equal(computed_syndrome, self.syndrome):
            return True, e_full

        return False, None

    def get_circuit_analysis(self):
        """Get comprehensive circuit analysis results"""
        total_oracle_gates = self.circuit_stats['oracle_gates'] * self.circuit_stats['total_iterations']
        total_diffuser_gates = self.circuit_stats['diffuser_gates'] * self.circuit_stats['total_iterations']

        return {
            'max_depth': self.max_circuit_depth,
            'total_gate_count': self.total_gate_count,
            'preprocessing': {
                'depth': self.circuit_stats['preprocessing_depth'],
                'gates': self.circuit_stats['preprocessing_gates']
            },
            'oracle': {
                'depth_per_iteration': self.circuit_stats['oracle_depth'],
                'gates_per_iteration': self.circuit_stats['oracle_gates'],
                'total_gates': total_oracle_gates
            },
            'diffuser': {
                'depth_per_iteration': self.circuit_stats['diffuser_depth'],
                'gates_per_iteration': self.circuit_stats['diffuser_gates'],
                'total_gates': total_diffuser_gates
            },
            'grover_iterations': self.circuit_stats['total_iterations'],
            'theoretical_complexity': {
                'space_size': 2**self.k,
                'estimated_solutions': self.estimate_solution_count(),
                'classical_complexity': f"O(2^{self.k})",
                'quantum_speedup': f"O(2^{self.k/2})"
            }
        }

    def decode(self, shots=2048, max_quantum_attempts=2):
        """Main quantum decoding function with comprehensive analysis"""
        print("Enhanced Quantum Lee-Brickell Decoding")
        print("="*50)
        print(f"Parameters: n={self.n}, k={self.k}, t={self.target_weight}, p={self.p_weight}")

        # Quantum preprocessing
        print("\nPhase 1: Quantum Preprocessing")
        if not self.quantum_preprocessing():
            print("❌ Quantum preprocessing failed")
            return None, self.get_circuit_analysis()

        print("✅ Quantum preprocessing successful")

        # Multiple quantum search attempts
        for attempt in range(max_quantum_attempts):
            print(f"\nPhase 2: Quantum Search (Attempt {attempt + 1})")

            try:
                qc, circuit_stats = self.quantum_search(shots)
                if qc is None:
                    continue

                # Execute quantum circuit
                simulator = AerSimulator()
                job = simulator.run(qc, shots=shots)
                result = job.result()
                counts = result.get_counts()

                print(f"Top measurement outcomes:")
                top_results = sorted(counts.items(), key=lambda x: x[1], reverse=True)[:5]
                for bitstring, count in top_results:
                    print(f"  {bitstring}: {count} ({100*count/shots:.1f}%)")

                # Solution verification
                print(f"\nPhase 3: Solution Verification")
                for bitstring, count in top_results:
                    is_valid, error_vector = self.verify_quantum_solution(bitstring[::-1])

                    if is_valid:
                        print(f"✅ Valid solution found!")
                        print(f"   Error vector: {error_vector}")
                        print(f"   Weight: {np.sum(error_vector)}")
                        return error_vector, self.get_circuit_analysis()

                print(f"❌ No valid solution in attempt {attempt + 1}")

            except Exception as e:
                print(f"❌ Quantum search failed: {e}")
                continue

        print(f"\n❌ All quantum attempts failed")
        return None, self.get_circuit_analysis()


def run_quantum_lb_decoder(n, k, t, p, shots=4096):
    """
    Run the Enhanced Quantum Lee-Brickell decoder

    Args:
        n: Code length
        k: Code dimension
        t: Target error weight
        p: Weight of e2 portion
        shots: Number of quantum measurements

    Returns:
        tuple: (decoded_error, circuit_analysis)
    """
    print(f"Quantum Lee-Brickell Decoder")
    print(f"Parameters: n={n}, k={k}, t={t}, p={p}")
    print("="*60)

    # Validate parameters
    if n <= k:
        raise ValueError("n must be greater than k")
    if t < 0 or t > n:
        raise ValueError("t must be between 0 and n")
    if p < 0 or p > k:
        raise ValueError("p must be between 0 and k")
    if p > t:
        raise ValueError("p cannot be greater than t")

    # Create and run decoder
    decoder = EnhancedQuantumLeeBrickell(n, k, t, p)
    decoded_error, circuit_analysis = decoder.decode(shots=shots)

    # Print circuit analysis
    print("\n" + "="*60)
    print("CIRCUIT COMPLEXITY ANALYSIS")
    print("="*60)

    analysis = circuit_analysis
    print(f"📊 Overall Circuit Metrics:")
    print(f"   Max Circuit Depth: {analysis['max_depth']}")
    print(f"   Total Gate Count: {analysis['total_gate_count']}")
    print(f"   Grover Iterations: {analysis['grover_iterations']}")

    print(f"\n🔧 Component Analysis:")
    print(f"   Preprocessing: {analysis['preprocessing']['gates']} gates, depth {analysis['preprocessing']['depth']}")
    print(f"   Oracle (per iter): {analysis['oracle']['gates_per_iteration']} gates, depth {analysis['oracle']['depth_per_iteration']}")
    print(f"   Diffuser (per iter): {analysis['diffuser']['gates_per_iteration']} gates, depth {analysis['diffuser']['depth_per_iteration']}")
    print(f"   Total Oracle gates: {analysis['oracle']['total_gates']}")
    print(f"   Total Diffuser gates: {analysis['diffuser']['total_gates']}")

    print(f"\n⚡ Theoretical Complexity:")
    print(f"   Search space: {analysis['theoretical_complexity']['space_size']:,}")
    print(f"   Estimated solutions: {analysis['theoretical_complexity']['estimated_solutions']}")
    print(f"   Classical complexity: {analysis['theoretical_complexity']['classical_complexity']}")
    print(f"   Quantum advantage: {analysis['theoretical_complexity']['quantum_speedup']}")

    return decoded_error, analysis


# Example usage and testing
if __name__ == "__main__":
    print("Enhanced Quantum Lee-Brickell Information Set Decoding")
    print("=" * 60)

    # Test cases with different parameters
    test_cases = [
        (7, 4, 2, 1),   # Small example
        (15, 11, 3, 2), # Medium example
        (31, 26, 4, 2), # Larger example (might be slow)
    ]

    for n, k, t, p in test_cases[:1]:  # Run first test case
        print(f"\nTesting parameters: n={n}, k={k}, t={t}, p={p}")
        try:
            decoded_error, analysis = run_quantum_lb_decoder(n, k, t, p, shots=2048)

            if decoded_error is not None:
                print(f"✅ Successfully decoded error of weight {np.sum(decoded_error)}")
            else:
                print(f"❌ Decoding failed for parameters n={n}, k={k}, t={t}, p={p}")

        except Exception as e:
            print(f"❌ Error with parameters n={n}, k={k}, t={t}, p={p}: {e}")

        print("-" * 60)

Enhanced Quantum Lee-Brickell Information Set Decoding

Testing parameters: n=7, k=4, t=2, p=1
Quantum Lee-Brickell Decoder
Parameters: n=7, k=4, t=2, p=1
Enhanced Quantum Lee-Brickell Decoding
Parameters: n=7, k=4, t=2, p=1

Phase 1: Quantum Preprocessing
Preprocessing circuit: 8 gates, depth 2


/tmp/ipython-input-587488890.py:95: DeprecationWarning: Treating CircuitInstruction as an iterable is deprecated legacy behavior since Qiskit 1.2, and will be removed in Qiskit 3.0. Instead, use the `operation`, `qubits` and `clbits` named attributes.
  gate_types[instruction[0].name] += 1


✅ Quantum preprocessing successful

Phase 2: Quantum Search (Attempt 1)
Search parameters:
  Total space: 2^4 = 16
  Estimated solutions: 1
  Grover iterations: 3
Oracle: 57 gates, depth 24
Diffuser: 19 gates, depth 7
Complete circuit: 236 gates, depth 95


/tmp/ipython-input-587488890.py:95: DeprecationWarning: Treating CircuitInstruction as an iterable is deprecated legacy behavior since Qiskit 1.2, and will be removed in Qiskit 3.0. Instead, use the `operation`, `qubits` and `clbits` named attributes.
  gate_types[instruction[0].name] += 1
/tmp/ipython-input-587488890.py:95: DeprecationWarning: Treating CircuitInstruction as an iterable is deprecated legacy behavior since Qiskit 1.2, and will be removed in Qiskit 3.0. Instead, use the `operation`, `qubits` and `clbits` named attributes.
  gate_types[instruction[0].name] += 1


Top measurement outcomes:
  0101: 142 (6.9%)
  0000: 137 (6.7%)
  1111: 136 (6.6%)
  0010: 135 (6.6%)
  0001: 135 (6.6%)

Phase 3: Solution Verification
❌ No valid solution in attempt 1

Phase 2: Quantum Search (Attempt 2)
Search parameters:
  Total space: 2^4 = 16
  Estimated solutions: 1
  Grover iterations: 3
Oracle: 57 gates, depth 24
Diffuser: 19 gates, depth 7
Complete circuit: 236 gates, depth 95


/tmp/ipython-input-587488890.py:95: DeprecationWarning: Treating CircuitInstruction as an iterable is deprecated legacy behavior since Qiskit 1.2, and will be removed in Qiskit 3.0. Instead, use the `operation`, `qubits` and `clbits` named attributes.
  gate_types[instruction[0].name] += 1
/tmp/ipython-input-587488890.py:95: DeprecationWarning: Treating CircuitInstruction as an iterable is deprecated legacy behavior since Qiskit 1.2, and will be removed in Qiskit 3.0. Instead, use the `operation`, `qubits` and `clbits` named attributes.
  gate_types[instruction[0].name] += 1


Top measurement outcomes:
  0010: 144 (7.0%)
  1011: 140 (6.8%)
  1010: 140 (6.8%)
  0111: 140 (6.8%)
  1100: 136 (6.6%)

Phase 3: Solution Verification
❌ No valid solution in attempt 2

❌ All quantum attempts failed

CIRCUIT COMPLEXITY ANALYSIS
📊 Overall Circuit Metrics:
   Max Circuit Depth: 95
   Total Gate Count: 236
   Grover Iterations: 3

🔧 Component Analysis:
   Preprocessing: 8 gates, depth 2
   Oracle (per iter): 57 gates, depth 24
   Diffuser (per iter): 19 gates, depth 7
   Total Oracle gates: 171
   Total Diffuser gates: 57

⚡ Theoretical Complexity:
   Search space: 16
   Estimated solutions: 1
   Classical complexity: O(2^4)
   Quantum advantage: O(2^2.0)
❌ Decoding failed for parameters n=7, k=4, t=2, p=1
------------------------------------------------------------


In [ ]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.library import QFT
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Statevector
from qiskit.circuit.library import MCXGate
import random
from math import pi, sqrt, floor, log2, ceil
from itertools import combinations
from collections import defaultdict

class EnhancedQuantumLeeBrickell:
    def __init__(self, n, k, t, p, H=None, syndrome=None):
        """
        Enhanced Quantum Lee-Brickell decoder with circuit analysis

        Args:
            n: Code length
            k: Code dimension
            t: Target error weight
            p: Weight of e2 portion
            H: Parity check matrix (optional - will generate if None)
            syndrome: Error syndrome (optional - will generate if None)
        """
        self.n = n
        self.k = k
        self.m = n - k
        self.target_weight = t
        self.p_weight = p
        self.e1_weight = t - p

        # Circuit analysis tracking
        self.max_circuit_depth = 0
        self.total_gate_count = 0
        self.circuit_stats = {
            'preprocessing_depth': 0,
            'preprocessing_gates': 0,
            'oracle_depth': 0,
            'oracle_gates': 0,
            'diffuser_depth': 0,
            'diffuser_gates': 0,
            'total_iterations': 0
        }

        # Generate or use provided matrices
        if H is None:
            self.H = self.generate_parity_check_matrix()
        else:
            if H.shape != (self.m, self.n):
                raise ValueError(f"H matrix shape {H.shape} doesn't match parameters ({self.m}, {self.n})")
            self.H = H

        if syndrome is None:
            self.syndrome = self.generate_test_syndrome()
        else:
            if len(syndrome) != self.m:
                raise ValueError(f"Syndrome length {len(syndrome)} doesn't match m={self.m}")
            self.syndrome = syndrome

        # Quantum preprocessing results
        self.X = None
        self.s_prime = None
        self.P = None
        self.preprocessing_success = False

    def generate_parity_check_matrix(self):
        """Generate a random parity check matrix in systematic form [I | P]"""
        np.random.seed(42)  # For reproducibility
        I = np.eye(self.m, dtype=int)
        P = np.random.randint(0, 2, (self.m, self.k))
        return np.hstack([I, P])

    def generate_test_syndrome(self):
        """Generate a test syndrome from a random error"""
        np.random.seed(123)
        test_error = np.zeros(self.n, dtype=int)

        # Place errors strategically
        error_positions = np.random.choice(self.n, min(self.target_weight, self.n), replace=False)
        test_error[error_positions] = 1

        return (self.H @ test_error) % 2

    def analyze_circuit_complexity(self, qc):
        """Analyze quantum circuit depth and gate count"""
        # Basic gate count
        gate_count = len(qc.data)

        # Calculate depth using Qiskit's built-in method
        depth = qc.depth()

        # Detailed gate analysis
        gate_types = defaultdict(int)
        for instruction in qc.data:
            gate_types[instruction[0].name] += 1

        return {
            'depth': depth,
            'gate_count': gate_count,
            'gate_types': dict(gate_types)
        }

    def quantum_preprocessing(self, max_attempts=10):
        """Quantum-enhanced preprocessing with circuit analysis"""
        preprocessing_circuit = self.create_preprocessing_circuit(max_attempts)

        # Analyze preprocessing circuit
        stats = self.analyze_circuit_complexity(preprocessing_circuit)
        self.circuit_stats['preprocessing_depth'] = stats['depth']
        self.circuit_stats['preprocessing_gates'] = stats['gate_count']

        print(f"Preprocessing circuit: {stats['gate_count']} gates, depth {stats['depth']}")

        # Simulate preprocessing
        simulator = AerSimulator()
        job = simulator.run(preprocessing_circuit, shots=1024)
        result = job.result()
        counts = result.get_counts()

        # Extract best permutation result
        success = self.extract_preprocessing_result(counts)
        self.preprocessing_success = success
        return success

    def create_preprocessing_circuit(self, max_attempts):
        """Create quantum circuit for preprocessing with superposition of permutations"""
        perm_qubits = max(1, ceil(log2(max_attempts)))
        perm_reg = QuantumRegister(perm_qubits, 'perm')
        result_reg = ClassicalRegister(perm_qubits, 'result')

        qc = QuantumCircuit(perm_reg, result_reg)

        # Create superposition over permutation choices
        qc.h(perm_reg)
        qc.measure(perm_reg, result_reg)

        return qc

    def extract_preprocessing_result(self, counts):
        """Extract the best preprocessing result from quantum measurements"""
        for bitstring, count in sorted(counts.items(), key=lambda x: x[1], reverse=True):
            perm_seed = int(bitstring, 2)
            if self.try_permutation(perm_seed):
                return True
        return False

    def try_permutation(self, seed):
        """Try a specific permutation for preprocessing"""
        random.seed(seed)
        perm_indices = list(range(self.n))
        random.shuffle(perm_indices)
        self.P = np.eye(self.n, dtype=int)[perm_indices]

        # Apply permutation and Gaussian elimination
        H_perm = self.H @ self.P
        return self.gaussian_elimination(H_perm)

    def gaussian_elimination(self, H_perm):
        """Perform Gaussian elimination to get systematic form"""
        H_work = H_perm.copy()
        pivot_cols = []

        for row in range(min(self.m, self.n)):
            pivot_col = None
            for col in range(self.n):
                if col not in pivot_cols and H_work[row, col] == 1:
                    pivot_col = col
                    break

            if pivot_col is None:
                continue

            pivot_cols.append(pivot_col)

            for other_row in range(self.m):
                if other_row != row and H_work[other_row, pivot_col] == 1:
                    H_work[other_row] = (H_work[other_row] + H_work[row]) % 2

        if len(pivot_cols) < self.m:
            return False

        # Extract systematic form components
        info_cols = [col for col in range(self.n) if col not in pivot_cols[:self.m]]

        if len(info_cols) != self.k:
            return False

        self.X = H_work[:self.m, info_cols]
        self.s_prime = self.syndrome.copy()
        return True

    def create_quantum_weight_oracle(self, qreg, target_weight, ancilla_reg):
        """Enhanced quantum weight checking oracle with complexity analysis"""
        qc = QuantumCircuit(qreg, ancilla_reg)
        n_qubits = len(qreg)

        if target_weight == 0:
            # All qubits must be |0⟩
            for i in range(n_qubits):
                qc.x(qreg[i])
            qc.mcx(qreg, ancilla_reg[0])
            for i in range(n_qubits):
                qc.x(qreg[i])

        elif target_weight == n_qubits:
            # All qubits must be |1⟩
            qc.mcx(qreg, ancilla_reg[0])

        else:
            # Use enumeration-based approach for practical implementation
            self.enumerate_weight_check(qc, qreg, target_weight, ancilla_reg[0])

        return qc

    def enumerate_weight_check(self, qc, qreg, target_weight, result_qubit):
        """Enumeration-based weight check with optimized gate count"""
        n_qubits = len(qreg)

        if n_qubits > 10:  # Practical limit for enumeration
            # For larger cases, use approximation or different strategy
            print(f"Warning: Large weight check ({n_qubits} qubits) - using simplified approach")
            return

        # Generate all combinations of target_weight positions
        for combo in combinations(range(n_qubits), target_weight):
            # Create temporary circuit for this combination
            temp_qc = QuantumCircuit(qreg, [result_qubit])

            # Flip qubits not in combination
            for i in range(n_qubits):
                if i not in combo:
                    temp_qc.x(qreg[i])

            # Multi-controlled X gate
            if len(qreg) == 1:
                temp_qc.cx(qreg[0], result_qubit)
            elif len(qreg) == 2:
                temp_qc.ccx(qreg[0], qreg[1], result_qubit)
            else:
                temp_qc.mcx(qreg, result_qubit)

            # Unflip qubits
            for i in range(n_qubits):
                if i not in combo:
                    temp_qc.x(qreg[i])

            qc.compose(temp_qc, inplace=True)

    def create_enhanced_oracle(self, e2_reg, ancilla_reg):
        """Enhanced oracle with both weight and syndrome constraints"""
        qc = QuantumCircuit(e2_reg, ancilla_reg)

        # Simplified oracle focusing on weight constraint
        weight_anc = ancilla_reg[0]

        # Check wt(e2) = p_weight
        weight_oracle = self.create_quantum_weight_oracle(e2_reg, self.p_weight, [weight_anc])
        qc.compose(weight_oracle, inplace=True)

        # Mark the solution (flip phase)
        qc.z(weight_anc)

        # Uncompute weight check
        weight_oracle_inv = weight_oracle.inverse()
        qc.compose(weight_oracle_inv, inplace=True)

        return qc

    def create_amplitude_amplification_diffuser(self, qreg):
        """Enhanced diffuser using amplitude amplification principles"""
        qc = QuantumCircuit(qreg)
        n_qubits = len(qreg)

        # Apply Hadamard gates
        qc.h(qreg)

        # Apply Z to |0...0⟩ state
        qc.x(qreg)

        # Multi-controlled Z gate
        if n_qubits == 1:
            qc.z(qreg[0])
        elif n_qubits == 2:
            qc.cz(qreg[0], qreg[1])
        else:
            qc.h(qreg[-1])
            qc.mcx(qreg[:-1], qreg[-1])
            qc.h(qreg[-1])

        qc.x(qreg)
        qc.h(qreg)

        return qc

    def estimate_solution_count(self):
        """Enhanced solution count estimation"""
        from math import comb

        # Theoretical upper bound
        if self.p_weight <= self.k:
            max_e2_combinations = comb(self.k, self.p_weight)
        else:
            max_e2_combinations = 0

        # Heuristic estimation
        if self.e1_weight <= self.m and max_e2_combinations > 0:
            prob_factor = comb(self.m, self.e1_weight) / (2**self.m) if self.m < 20 else 0.01
            estimated_solutions = max(1, int(max_e2_combinations * prob_factor))
        else:
            estimated_solutions = max(1, max_e2_combinations // 10)

        return min(estimated_solutions, max_e2_combinations)

    def adaptive_grover_iterations(self, total_space, estimated_solutions):
        """Adaptive iteration count for Grover's algorithm"""
        if estimated_solutions == 0:
            return 3

        # Optimal Grover iterations
        if estimated_solutions >= total_space:
            return 1

        optimal = int(pi * sqrt(total_space / estimated_solutions) / 4)

        # Conservative bounds
        min_iter = max(1, optimal // 2)
        max_iter = min(optimal * 2, 20)  # Safety limit

        return min(max_iter, max(min_iter, 3))

    def quantum_search(self, shots=2048):
        """Enhanced quantum search with comprehensive circuit analysis"""
        if not self.preprocessing_success:
            print("Preprocessing failed - cannot proceed with quantum search")
            return None, None

        # Create quantum registers
        e2_reg = QuantumRegister(self.k, 'e2')
        ancilla_reg = QuantumRegister(max(3, self.k), 'anc')  # Simplified ancilla
        result_reg = ClassicalRegister(self.k, 'result')

        qc = QuantumCircuit(e2_reg, ancilla_reg, result_reg)

        # Initialize superposition
        qc.h(e2_reg)

        # Calculate adaptive iteration count
        total_space = 2**self.k
        estimated_solutions = self.estimate_solution_count()
        iterations = self.adaptive_grover_iterations(total_space, estimated_solutions)

        self.circuit_stats['total_iterations'] = iterations

        print(f"Search parameters:")
        print(f"  Total space: 2^{self.k} = {total_space}")
        print(f"  Estimated solutions: {estimated_solutions}")
        print(f"  Grover iterations: {iterations}")

        # Create and analyze oracle
        oracle = self.create_enhanced_oracle(e2_reg, ancilla_reg)
        oracle_stats = self.analyze_circuit_complexity(oracle)
        self.circuit_stats['oracle_depth'] = oracle_stats['depth']
        self.circuit_stats['oracle_gates'] = oracle_stats['gate_count']

        # Create and analyze diffuser
        diffuser = self.create_amplitude_amplification_diffuser(e2_reg)
        diffuser_stats = self.analyze_circuit_complexity(diffuser)
        self.circuit_stats['diffuser_depth'] = diffuser_stats['depth']
        self.circuit_stats['diffuser_gates'] = diffuser_stats['gate_count']

        print(f"Oracle: {oracle_stats['gate_count']} gates, depth {oracle_stats['depth']}")
        print(f"Diffuser: {diffuser_stats['gate_count']} gates, depth {diffuser_stats['depth']}")

        # Grover iterations
        for i in range(iterations):
            qc.compose(oracle, inplace=True)
            qc.compose(diffuser, inplace=True)

        # Measurement
        qc.measure(e2_reg, result_reg)

        # Final circuit analysis
        final_stats = self.analyze_circuit_complexity(qc)
        self.max_circuit_depth = final_stats['depth']
        self.total_gate_count = final_stats['gate_count']

        print(f"Complete circuit: {final_stats['gate_count']} gates, depth {final_stats['depth']}")

        return qc, final_stats

    def verify_quantum_solution(self, e2_bitstring):
        """Verify a candidate solution from quantum measurement"""
        e2 = np.array([int(b) for b in e2_bitstring], dtype=int)

        # Check e2 weight
        if np.sum(e2) != self.p_weight:
            return False, None

        # Compute e1
        e1 = (self.s_prime + self.X @ e2) % 2

        # Check e1 weight
        if np.sum(e1) != self.e1_weight:
            return False, None

        # Reconstruct full error
        e_systematic = np.concatenate([e1, e2])
        e_full = self.P.T @ e_systematic

        # Final verification
        computed_syndrome = (self.H @ e_full) % 2
        if np.array_equal(computed_syndrome, self.syndrome):
            return True, e_full

        return False, None

    def get_circuit_analysis(self):
        """Get comprehensive circuit analysis results"""
        total_oracle_gates = self.circuit_stats['oracle_gates'] * self.circuit_stats['total_iterations']
        total_diffuser_gates = self.circuit_stats['diffuser_gates'] * self.circuit_stats['total_iterations']

        return {
            'max_depth': self.max_circuit_depth,
            'total_gate_count': self.total_gate_count,
            'preprocessing': {
                'depth': self.circuit_stats['preprocessing_depth'],
                'gates': self.circuit_stats['preprocessing_gates']
            },
            'oracle': {
                'depth_per_iteration': self.circuit_stats['oracle_depth'],
                'gates_per_iteration': self.circuit_stats['oracle_gates'],
                'total_gates': total_oracle_gates
            },
            'diffuser': {
                'depth_per_iteration': self.circuit_stats['diffuser_depth'],
                'gates_per_iteration': self.circuit_stats['diffuser_gates'],
                'total_gates': total_diffuser_gates
            },
            'grover_iterations': self.circuit_stats['total_iterations'],
            'theoretical_complexity': {
                'space_size': 2**self.k,
                'estimated_solutions': self.estimate_solution_count(),
                'classical_complexity': f"O(2^{self.k})",
                'quantum_speedup': f"O(2^{self.k/2})"
            }
        }

    def decode(self, shots=2048, max_quantum_attempts=2):
        """Main quantum decoding function with comprehensive analysis"""
        print("Enhanced Quantum Lee-Brickell Decoding")
        print("="*50)
        print(f"Parameters: n={self.n}, k={self.k}, t={self.target_weight}, p={self.p_weight}")

        # Quantum preprocessing
        print("\nPhase 1: Quantum Preprocessing")
        if not self.quantum_preprocessing():
            print("❌ Quantum preprocessing failed")
            return None, self.get_circuit_analysis()

        print("✅ Quantum preprocessing successful")

        # Multiple quantum search attempts
        for attempt in range(max_quantum_attempts):
            print(f"\nPhase 2: Quantum Search (Attempt {attempt + 1})")

            try:
                qc, circuit_stats = self.quantum_search(shots)
                if qc is None:
                    continue

                # Execute quantum circuit
                simulator = AerSimulator()
                job = simulator.run(qc, shots=shots)
                result = job.result()
                counts = result.get_counts()

                print(f"Top measurement outcomes:")
                top_results = sorted(counts.items(), key=lambda x: x[1], reverse=True)[:5]
                for bitstring, count in top_results:
                    print(f"  {bitstring}: {count} ({100*count/shots:.1f}%)")

                # Solution verification
                print(f"\nPhase 3: Solution Verification")
                for bitstring, count in top_results:
                    is_valid, error_vector = self.verify_quantum_solution(bitstring[::-1])

                    if is_valid:
                        print(f"✅ Valid solution found!")
                        print(f"   Error vector: {error_vector}")
                        print(f"   Weight: {np.sum(error_vector)}")
                        return error_vector, self.get_circuit_analysis()

                print(f"❌ No valid solution in attempt {attempt + 1}")

            except Exception as e:
                print(f"❌ Quantum search failed: {e}")
                continue

        print(f"\n❌ All quantum attempts failed")
        return None, self.get_circuit_analysis()


def run_quantum_lb_decoder(n, k, t, p, shots=4096):
    """
    Run the Enhanced Quantum Lee-Brickell decoder

    Args:
        n: Code length
        k: Code dimension
        t: Target error weight
        p: Weight of e2 portion
        shots: Number of quantum measurements

    Returns:
        tuple: (decoded_error, circuit_analysis)
    """
    print(f"Quantum Lee-Brickell Decoder")
    print(f"Parameters: n={n}, k={k}, t={t}, p={p}")
    print("="*60)

    # Validate parameters
    if n <= k:
        raise ValueError("n must be greater than k")
    if t < 0 or t > n:
        raise ValueError("t must be between 0 and n")
    if p < 0 or p > k:
        raise ValueError("p must be between 0 and k")
    if p > t:
        raise ValueError("p cannot be greater than t")

    # Create and run decoder
    decoder = EnhancedQuantumLeeBrickell(n, k, t, p)
    decoded_error, circuit_analysis = decoder.decode(shots=shots)

    # Print circuit analysis
    print("\n" + "="*60)
    print("CIRCUIT COMPLEXITY ANALYSIS")
    print("="*60)

    analysis = circuit_analysis
    print(f"📊 Overall Circuit Metrics:")
    print(f"   Max Circuit Depth: {analysis['max_depth']}")
    print(f"   Total Gate Count: {analysis['total_gate_count']}")
    print(f"   Grover Iterations: {analysis['grover_iterations']}")

    print(f"\n🔧 Component Analysis:")
    print(f"   Preprocessing: {analysis['preprocessing']['gates']} gates, depth {analysis['preprocessing']['depth']}")
    print(f"   Oracle (per iter): {analysis['oracle']['gates_per_iteration']} gates, depth {analysis['oracle']['depth_per_iteration']}")
    print(f"   Diffuser (per iter): {analysis['diffuser']['gates_per_iteration']} gates, depth {analysis['diffuser']['depth_per_iteration']}")
    print(f"   Total Oracle gates: {analysis['oracle']['total_gates']}")
    print(f"   Total Diffuser gates: {analysis['diffuser']['total_gates']}")

    print(f"\n⚡ Theoretical Complexity:")
    print(f"   Search space: {analysis['theoretical_complexity']['space_size']:,}")
    print(f"   Estimated solutions: {analysis['theoretical_complexity']['estimated_solutions']}")
    print(f"   Classical complexity: {analysis['theoretical_complexity']['classical_complexity']}")
    print(f"   Quantum advantage: {analysis['theoretical_complexity']['quantum_speedup']}")

    return decoded_error, analysis


def get_user_input():
    """Get user input for quantum Lee-Brickell parameters"""
    print("Enhanced Quantum Lee-Brickell Information Set Decoding")
    print("=" * 60)
    print("Please enter the following parameters:")

    while True:
        try:
            # Get basic parameters
            n = int(input("📏 Code length (n): "))
            k = int(input("📐 Code dimension (k): "))
            t = int(input("⚖️  Target error weight (t): "))
            p = int(input("🎯 Weight of e2 portion (p): "))

            # Validate parameters
            if n <= 0:
                print("❌ Code length n must be positive")
                continue
            if k <= 0 or k >= n:
                print("❌ Code dimension k must be positive and less than n")
                continue
            if t < 0 or t > n:
                print("❌ Target error weight t must be between 0 and n")
                continue
            if p < 0 or p > k:
                print("❌ Weight p must be between 0 and k")
                continue
            if p > t:
                print("❌ Weight p cannot be greater than target weight t")
                continue

            # Get optional parameters
            shots = input(f"🔢 Number of quantum shots (default: 2048): ").strip()
            shots = int(shots) if shots else 2048

            if shots <= 0:
                print("❌ Number of shots must be positive")
                continue

            attempts = input(f"🔄 Max quantum attempts (default: 2): ").strip()
            attempts = int(attempts) if attempts else 2

            if attempts <= 0:
                print("❌ Number of attempts must be positive")
                continue

            return n, k, t, p, shots, attempts

        except ValueError:
            print("❌ Please enter valid integer values")
            continue
        except KeyboardInterrupt:
            print("\n👋 Goodbye!")
            return None

def interactive_mode():
    """Run the decoder in interactive mode with user input"""
    user_input = get_user_input()
    if user_input is None:
        return

    n, k, t, p, shots, max_attempts = user_input

    print(f"\n🚀 Starting quantum decoding with parameters:")
    print(f"   Code: [{n},{k}] (length, dimension)")
    print(f"   Error weight: t={t}, p={p}")
    print(f"   Quantum shots: {shots}")
    print(f"   Max attempts: {max_attempts}")
    print("=" * 60)

    try:
        # Create and run decoder
        decoder = EnhancedQuantumLeeBrickell(n, k, t, p)
        decoded_error, circuit_analysis = decoder.decode(shots=shots, max_quantum_attempts=max_attempts)

        # Print results
        print("\n" + "="*60)
        print("🏁 DECODING RESULTS")
        print("="*60)

        if decoded_error is not None:
            print("🎉 DECODING SUCCESSFUL!")
            print(f"   📍 Error vector: {decoded_error}")
            print(f"   ⚖️  Decoded weight: {np.sum(decoded_error)}")
            print(f"   🎯 Target weight: {t}")
            print(f"   ✅ Weight match: {np.sum(decoded_error) == t}")

            # Verify syndrome
            computed_syndrome = (decoder.H @ decoded_error) % 2
            syndrome_match = np.array_equal(computed_syndrome, decoder.syndrome)
            print(f"   🔍 Syndrome verification: {'✅ PASSED' if syndrome_match else '❌ FAILED'}")
        else:
            print("❌ DECODING FAILED")
            print("💡 Suggestions for improvement:")
            print("   • Increase number of quantum shots")
            print("   • Try different weight parameter p")
            print("   • Increase max quantum attempts")

        # Print circuit analysis
        print("\n" + "="*60)
        print("⚙️  CIRCUIT COMPLEXITY ANALYSIS")
        print("="*60)

        analysis = circuit_analysis
        print(f"📊 Overall Metrics:")
        print(f"   🏗️  Max Circuit Depth: {analysis['max_depth']}")
        print(f"   🔧 Total Gate Count: {analysis['total_gate_count']:,}")
        print(f"   🔄 Grover Iterations: {analysis['grover_iterations']}")

        print(f"\n🧩 Component Breakdown:")
        print(f"   🔄 Preprocessing: {analysis['preprocessing']['gates']} gates, depth {analysis['preprocessing']['depth']}")
        print(f"   🎯 Oracle (per iteration): {analysis['oracle']['gates_per_iteration']} gates, depth {analysis['oracle']['depth_per_iteration']}")
        print(f"   🔀 Diffuser (per iteration): {analysis['diffuser']['gates_per_iteration']} gates, depth {analysis['diffuser']['depth_per_iteration']}")
        print(f"   📊 Total Oracle gates: {analysis['oracle']['total_gates']:,}")
        print(f"   📊 Total Diffuser gates: {analysis['diffuser']['total_gates']:,}")

        print(f"\n⚡ Complexity Analysis:")
        print(f"   🌌 Search space size: {analysis['theoretical_complexity']['space_size']:,}")
        print(f"   🎯 Estimated solutions: {analysis['theoretical_complexity']['estimated_solutions']}")
        print(f"   🐌 Classical complexity: {analysis['theoretical_complexity']['classical_complexity']}")
        print(f"   ⚡ Quantum speedup: {analysis['theoretical_complexity']['quantum_speedup']}")

        # Ask if user wants to run again
        print("\n" + "="*60)
        run_again = input("🔄 Would you like to try different parameters? (y/n): ").strip().lower()
        if run_again in ['y', 'yes']:
            interactive_mode()
        else:
            print("👋 Thank you for using the Quantum Lee-Brickell Decoder!")

    except Exception as e:
        print(f"❌ An error occurred: {e}")
        print("Please check your parameters and try again.")

# Example usage and testing
if __name__ == "__main__":
    # Run in interactive mode
    interactive_mode()

    # Uncomment below for automated testing
    """
    print("\n🧪 AUTOMATED TESTING")
    print("=" * 60)

    # Test cases with different parameters
    test_cases = [
        (7, 4, 2, 1),   # Small example
        (15, 11, 3, 2), # Medium example
    ]

    for n, k, t, p in test_cases:
        print(f"\nTesting parameters: n={n}, k={k}, t={t}, p={p}")
        try:
            decoded_error, analysis = run_quantum_lb_decoder(n, k, t, p, shots=1024)

            if decoded_error is not None:
                print(f"✅ Successfully decoded error of weight {np.sum(decoded_error)}")
            else:
                print(f"❌ Decoding failed for parameters n={n}, k={k}, t={t}, p={p}")

        except Exception as e:
            print(f"❌ Error with parameters n={n}, k={k}, t={t}, p={p}: {e}")

        print("-" * 60)
    """

Enhanced Quantum Lee-Brickell Information Set Decoding
Please enter the following parameters:
📏 Code length (n): 20
📐 Code dimension (k): 10
⚖️  Target error weight (t): 8
🎯 Weight of e2 portion (p): 4
🔢 Number of quantum shots (default: 2048): 2048
🔄 Max quantum attempts (default: 2): 2

🚀 Starting quantum decoding with parameters:
   Code: [20,10] (length, dimension)
   Error weight: t=8, p=4
   Quantum shots: 2048
   Max attempts: 2
Enhanced Quantum Lee-Brickell Decoding
Parameters: n=20, k=10, t=8, p=4

Phase 1: Quantum Preprocessing
Preprocessing circuit: 8 gates, depth 2
✅ Quantum preprocessing successful

Phase 2: Quantum Search (Attempt 1)
Search parameters:
  Total space: 2^10 = 1024
  Estimated solutions: 43
  Grover iterations: 3


/tmp/ipython-input-2055670589.py:95: DeprecationWarning: Treating CircuitInstruction as an iterable is deprecated legacy behavior since Qiskit 1.2, and will be removed in Qiskit 3.0. Instead, use the `operation`, `qubits` and `clbits` named attributes.
  gate_types[instruction[0].name] += 1
/tmp/ipython-input-2055670589.py:95: DeprecationWarning: Treating CircuitInstruction as an iterable is deprecated legacy behavior since Qiskit 1.2, and will be removed in Qiskit 3.0. Instead, use the `operation`, `qubits` and `clbits` named attributes.
  gate_types[instruction[0].name] += 1
/tmp/ipython-input-2055670589.py:95: DeprecationWarning: Treating CircuitInstruction as an iterable is deprecated legacy behavior since Qiskit 1.2, and will be removed in Qiskit 3.0. Instead, use the `operation`, `qubits` and `clbits` named attributes.
  gate_types[instruction[0].name] += 1


Oracle: 5461 gates, depth 1260
Diffuser: 43 gates, depth 7
Complete circuit: 16532 gates, depth 3803
Top measurement outcomes:
  1111101010: 9 (0.4%)
  1000011111: 9 (0.4%)
  0010011000: 8 (0.4%)
  0001011000: 8 (0.4%)
  1011100100: 7 (0.3%)

Phase 3: Solution Verification
❌ No valid solution in attempt 1

Phase 2: Quantum Search (Attempt 2)
Search parameters:
  Total space: 2^10 = 1024
  Estimated solutions: 43
  Grover iterations: 3
Oracle: 5461 gates, depth 1260
Diffuser: 43 gates, depth 7


/tmp/ipython-input-2055670589.py:95: DeprecationWarning: Treating CircuitInstruction as an iterable is deprecated legacy behavior since Qiskit 1.2, and will be removed in Qiskit 3.0. Instead, use the `operation`, `qubits` and `clbits` named attributes.
  gate_types[instruction[0].name] += 1
/tmp/ipython-input-2055670589.py:95: DeprecationWarning: Treating CircuitInstruction as an iterable is deprecated legacy behavior since Qiskit 1.2, and will be removed in Qiskit 3.0. Instead, use the `operation`, `qubits` and `clbits` named attributes.
  gate_types[instruction[0].name] += 1


Complete circuit: 16532 gates, depth 3803
Top measurement outcomes:
  1010110110: 8 (0.4%)
  0000110001: 7 (0.3%)
  0010100100: 7 (0.3%)
  1011100111: 7 (0.3%)
  1011110011: 7 (0.3%)

Phase 3: Solution Verification
❌ No valid solution in attempt 2

❌ All quantum attempts failed

🏁 DECODING RESULTS
❌ DECODING FAILED
💡 Suggestions for improvement:
   • Increase number of quantum shots
   • Try different weight parameter p
   • Increase max quantum attempts

⚙️  CIRCUIT COMPLEXITY ANALYSIS
📊 Overall Metrics:
   🏗️  Max Circuit Depth: 3803
   🔧 Total Gate Count: 16,532
   🔄 Grover Iterations: 3

🧩 Component Breakdown:
   🔄 Preprocessing: 8 gates, depth 2
   🎯 Oracle (per iteration): 5461 gates, depth 1260
   🔀 Diffuser (per iteration): 43 gates, depth 7
   📊 Total Oracle gates: 16,383
   📊 Total Diffuser gates: 129

⚡ Complexity Analysis:
   🌌 Search space size: 1,024
   🎯 Estimated solutions: 43
   🐌 Classical complexity: O(2^10)
   ⚡ Quantum speedup: O(2^5.0)

🔄 Would you like to try dif